In [2]:
#import requests
import pandas as pd
import psycopg2
from sqlalchemy.types import String
from sqlalchemy import create_engine

In [3]:
file1 = pd.read_csv('Data Model/nsd_100_bss_quart.csv',  parse_dates=True)
file1 = file1.drop('Unnamed: 0', axis='columns')
file1 = file1.set_index(keys=['ticker','date'])

file2 = pd.read_csv('Data Model/nsd_100_cfs_quart.csv',  parse_dates=True)
file2 = file2.drop('Unnamed: 0', axis='columns')
file2 = file2.set_index(keys=['ticker','date'])

file3 = pd.read_csv('Data Model/nsd_100_is_quart.csv',  parse_dates=True)
file3 = file3.drop('Unnamed: 0', axis='columns')
file3 = file3.set_index(keys=['ticker','date'])

# Join 3 fin statements
joined_data = file1.join(file2, how='outer')
joined_data = joined_data.join(file3, how='outer')
joined_data.sort_index(level=1, inplace=True, ascending=False)
print(joined_data)

                   Cash and cash equivalents  Short-term investments  \
ticker date                                                            
WDC    2020-04-03               2.943000e+09            0.000000e+00   
VRTX   2020-03-31               3.593412e+09            5.969840e+08   
VRSN   2020-03-31               4.986930e+08            6.399390e+08   
VRSK   2020-03-31               2.044000e+08            0.000000e+00   
UAL    2020-03-31               3.442000e+09            1.779000e+09   
...                                      ...                     ...   
PEP    2009-06-13               2.214000e+09            2.100000e+08   
MU     2009-06-04               1.306000e+09            0.000000e+00   
WBA    2009-05-31               2.300000e+09            1.000000e+08   
PAYX   2009-05-31               4.727690e+08            1.971000e+07   
CTAS   2009-05-31               1.297450e+08            1.203930e+08   

                   Cash and short-term investments   Receivable

In [19]:
file4 = pd.read_csv('ent_val_nsd100full.csv', parse_dates=True)
file4 = file4.drop(['Unnamed: 0','Stock Price', 'Market Capitalization', '- Cash & Cash Equivalents', '+ Total Debt', 'Enterprise Value'], axis='columns')
file4 = file4.set_index(keys=['ticker','date'])

In [9]:
# Question 1. Return on Assets (ROA)
piotroski_dfs1 = pd.DataFrame(columns=['ticker','date','score1'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:4+item]
        
        # Calculate if no data is missing
        if((temp33['Net Income'].all()) and (len(temp33.index) == 4)):
            
            # Calculating 1st score
            mean_income = temp33['Net Income'].sum()
            
            if(mean_income >= 0):
                piotroski_dfs1 = piotroski_dfs1.append({'ticker':ticker, 'date':new_df.index[item][1], 'score1':1}, ignore_index=True)
            else:
                piotroski_dfs1 = piotroski_dfs1.append({'ticker':ticker, 'date':new_df.index[item][1], 'score1':0}, ignore_index=True)
        else:
            piotroski_dfs1 = piotroski_dfs1.append({'ticker':ticker, 'date':new_df.index[item][1], 'score1':None}, ignore_index=True)

In [10]:
# Question 2. Cash Flow Return on Assets (CFROA)
piotroski_dfs2 = pd.DataFrame(columns=['ticker','date','score2'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:4+item]
        
        # Calculate if no data is missing
        if((temp33['Operating Cash Flow'].all()) and (len(temp33.index) == 4)):
            
            # Calculating 2st score
            mean_ocf = temp33['Operating Cash Flow'].sum()
            
            if(mean_ocf >= 0):
                piotroski_dfs2 = piotroski_dfs2.append({'ticker':ticker, 'date':new_df.index[item][1], 'score2':1}, ignore_index=True)
            else:
                piotroski_dfs2 = piotroski_dfs2.append({'ticker':ticker, 'date':new_df.index[item][1], 'score2':0}, ignore_index=True)
        else:
            piotroski_dfs2 = piotroski_dfs2.append({'ticker':ticker, 'date':new_df.index[item][1], 'score2':None}, ignore_index=True)

In [13]:
# Question 3. Change in Return on Assets

piotroski_dfs3 = pd.DataFrame(columns=['ticker','date','score3'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:9+item]
        
        # Calculate if no data is missing
        if((temp33['Net Income'].all()) and (len(temp33.index) == 9) 
           and (temp33['Total assets'][8] != 0) and (temp33['Total assets'][4] != 0)):
            
            # Calculating 3rd score
            roa_this_year = temp33['Net Income'][0:4].sum() / temp33['Total assets'][4]
            roa_last_year = temp33['Net Income'][4:8].sum() / temp33['Total assets'][8]
            
            print(new_df.index[item])
            print(roa_this_year)
            print(roa_last_year)
            
            if(roa_this_year > roa_last_year):
                piotroski_dfs3 = piotroski_dfs3.append({'ticker':ticker, 'date':new_df.index[item][1], 'score3':1}, ignore_index=True)
            else:
                piotroski_dfs3 = piotroski_dfs3.append({'ticker':ticker, 'date':new_df.index[item][1], 'score3':0}, ignore_index=True)
        else:
            piotroski_dfs3 = piotroski_dfs3.append({'ticker':ticker, 'date':new_df.index[item][1], 'score3':None}, ignore_index=True)

('AAPL', '2020-03-28')
0.16729629997836246
0.1555637792447388
('AAPL', '2019-12-28')
0.15393116218335165
0.14609605844727308
('AAPL', '2019-09-28')
0.15108619864652403
0.15861440534585247
('AAPL', '2019-06-29')
0.159494497375407
0.1625851384667978
('AAPL', '2019-03-30')
0.1555637792447388
0.15938086640440974
('AAPL', '2018-12-29')
0.14609605844727308
0.15257850885272436
('AAPL', '2018-09-29')
0.15861440534585247
0.15030495576431677
('AAPL', '2018-06-30')
0.1625851384667978
0.15265279677489021
('AAPL', '2018-03-31')
0.15938086640440974
0.14979837983208694
('AAPL', '2017-12-30')
0.15257850885272436
0.1541747930333738
('AAPL', '2017-09-30')
0.15030495576431677
0.15728159350589888
('AAPL', '2017-07-01')
0.15265279677489021
0.17498380016913723
('AAPL', '2017-04-01')
0.14979837983208694
0.194024365031356
('AAPL', '2016-12-31')
0.1541747930333738
0.20516315761338558
('AAPL', '2016-09-24')
0.15728159350589888
0.23030637640776574
('AAPL', '2016-06-25')
0.17498380016913723
0.22801096530648932
('

('ADSK', '2016-10-31')
-0.08542145593869732
-0.059491639925688226
('ADSK', '2016-07-31')
-0.07113706990153132
-0.04128928009525024
('ADSK', '2016-04-30')
-0.10942440168345233
0.015617605300520587
('ADSK', '2016-01-31')
-0.06725955472343197
0.01780195865070729
('ADSK', '2015-10-31')
-0.059491639925688226
0.02874002082610205
('ADSK', '2015-07-31')
-0.04128928009525024
0.040513342646745436
('ADSK', '2015-04-30')
0.015617605300520587
0.04697298179359862
('ADSK', '2015-01-31')
0.01780195865070729
0.053105561229226626
('ADSK', '2014-10-31')
0.02874002082610205
0.07272833313892453
('ADSK', '2014-07-31')
0.040513342646745436
0.06552326786930893
('ADSK', '2014-04-30')
0.04697298179359862
0.06756104914078986
('ADSK', '2014-01-31')
0.053105561229226626
0.07664663238118842
('ADSK', '2013-10-31')
0.07272833313892453
0.08149750415973378
('ADSK', '2013-07-31')
0.06552326786930893
0.09694992769949894
('ADSK', '2013-04-30')
0.06756104914078986
0.10161256977465372
('ADSK', '2013-01-31')
0.07664663238118

('AMD', '2011-07-02')
0.16710393541876892
0.180237245191754
('AMGN', '2020-03-31')
0.11992749660140319
0.11347029396886066
('AMGN', '2019-12-31')
0.11807395808238978
0.10498536658578683
('AMGN', '2019-09-30')
0.11980752379962277
0.027411584568846396
('AMGN', '2019-06-30')
0.11757579339282548
0.02970334351087489
('AMGN', '2019-03-31')
0.11347029396886066
0.028499139503223653
('AMGN', '2018-12-31')
0.10498536658578683
0.025494035503568394
('AMGN', '2018-09-30')
0.027411584568846396
0.10464491362763916
('AMGN', '2018-06-30')
0.02970334351087489
0.10830650183514197
('AMGN', '2018-03-31')
0.028499139503223653
0.105077480164013
('AMGN', '2017-12-31')
0.025494035503568394
0.10788532468984018
('AMGN', '2017-09-30')
0.10464491362763916
0.10556707342525985
('AMGN', '2017-06-30')
0.10830650183514197
0.10438287295145277
('AMGN', '2017-03-31')
0.105077480164013
0.1046525118923309
('AMGN', '2016-12-31')
0.10788532468984018
0.10055210189975221
('AMGN', '2016-09-30')
0.10556707342525985
0.090893677145

('AVGO', '2017-01-29')
-0.17416720794284124
0.12994297466579416
('AVGO', '2016-10-30')
-0.16538278649548263
0.13001620436564673
('AVGO', '2016-07-31')
-0.06788145774929916
0.10426817384525433
('AVGO', '2016-05-01')
-0.013292821876186859
0.18142195587033505
('AVGO', '2016-01-31')
0.12994297466579416
0.1382488479262673
('AVGO', '2015-11-01')
0.13001620436564673
0.07701317715959004
('AVGO', '2015-08-02')
0.10426817384525433
0.09363295880149813
('AVGO', '2015-05-03')
0.18142195587033505
0.19940769990128332
('AVGO', '2015-02-01')
0.1382488479262673
0.19398340248962656
('AVGO', '2014-11-02')
0.07701317715959004
0.1928721174004193
('AVGO', '2014-08-03')
0.09363295880149813
0.1985267034990792
('AVGO', '2014-05-04')
0.19940769990128332
0.2075833014170816
('AVGO', '2014-02-02')
0.19398340248962656
0.23036006546644844
('AVGO', '2013-11-03')
0.1928721174004193
0.2301717089125102
('AVGO', '2013-08-04')
0.1985267034990792
0.2454905411350638
('AVGO', '2013-05-05')
0.2075833014170816
0.255856683509416

('CDW', '2019-12-31')
0.10279448079579223
0.0924302101601357
('CDW', '2019-09-30')
0.09633375816904846
0.09559548283533752
('CDW', '2019-06-30')
0.09447347239464673
0.08865882753919564
('CDW', '2019-03-31')
0.09517779137437926
0.08744298640530208
('CDW', '2018-12-31')
0.0924302101601357
0.07526912670542858
('CDW', '2018-09-30')
0.09559548283533752
0.06363690054334988
('CDW', '2018-06-30')
0.08865882753919564
0.06210891563466368
('CDW', '2018-03-31')
0.08744298640530208
0.06016761190252906
('CDW', '2017-12-31')
0.07526912670542858
0.06282474501502525
('CDW', '2017-09-30')
0.06363690054334988
0.06089149299117407
('CDW', '2017-06-30')
0.06210891563466368
0.06966662400819043
('CDW', '2017-03-31')
0.06016761190252906
0.06999967151726177
('CDW', '2016-12-31')
0.06282474501502525
0.06608305054181215
('CDW', '2016-09-30')
0.06089149299117407
0.05986180698824377
('CDW', '2016-06-30')
0.06966662400819043
0.044712421219790584
('CDW', '2016-03-31')
0.06999967151726177
0.04206342494714588
('CDW', '

...

('REGN', '2016-03-31')
0.1728613507494094
0.1100025886426066
('REGN', '2015-12-31')
0.16427800105738197
0.1179506833755053
('REGN', '2015-09-30')
0.15810958868209204
0.12676189159658613
('REGN', '2015-06-30')
0.13326088126483257
0.16663029552290143
('REGN', '2015-03-31')
0.1100025886426066
0.17176245321832445
('REGN', '2014-12-31')
0.1179506833755053
0.2039721411782801
('REGN', '2014-09-30')
0.12676189159658613
0.4805225774788256
('REGN', '2014-06-30')
0.16663029552290143
0.5819039451114922
('REGN', '2014-03-31')
0.17176245321832445
0.6202913879032025
('REGN', '2013-12-31')
0.2039721411782801
0.5668469601075263
('REGN', '2013-09-30')
0.4805225774788256
0.23032742609337853
('REGN', '2013-06-30')
0.5819039451114922
-0.02618678261932782
('REGN', '2013-03-31')
0.6202913879032025
-0.15513644787781358
('REGN', '2012-12-31')
0.5668469601075263
-0.20355561430176458
('REGN', '2012-09-30')
0.23032742609337853
-0.19408700671559675
('REGN', '2012-06-30')
-0.02618678261932782
-0.1953149406620704
('

('SNPS', '2020-01-31')
0.07611554348137894
0.10314262282666926
('SNPS', '2019-10-31')
0.08662044453816434
0.08014915089909706
('SNPS', '2019-07-31')
0.1080168725405356
0.010232193020562467
('SNPS', '2019-04-30')
0.10623854308886484
0.01746071293856207
('SNPS', '2019-01-31')
0.10314262282666926
0.008792128951224618
('SNPS', '2018-10-31')
0.08014915089909706
0.026059825985403687
('SNPS', '2018-07-31')
0.010232193020562467
0.06446576674041539
('SNPS', '2018-04-30')
0.01746071293856207
0.05584981097678008
('SNPS', '2018-01-31')
0.008792128951224618
0.0617962251051759
('SNPS', '2017-10-31')
0.026059825985403687
0.05288145106197526
('SNPS', '2017-07-31')
0.06446576674041539
0.04923140461835473
('SNPS', '2017-04-30')
0.05584981097678008
0.049317023561199595
('SNPS', '2017-01-31')
0.0617962251051759
0.04735381242373748
('SNPS', '2016-10-31')
0.05288145106197526
0.04731107681103064
('SNPS', '2016-07-31')
0.04923140461835473
0.0525040105136956
('SNPS', '2016-04-30')
0.049317023561199595
0.054576

('TSLA', '2012-12-31')
-0.5553495139099135
-0.7857190959433489
('TSLA', '2012-09-30')
-0.5537607997143877
-0.7555451702196498
('TSLA', '2012-06-30')
-0.5293513166345536
-1.6427075026693878
('TSLA', '2012-03-31')
-0.8453064040521595
nan
('TSLA', '2011-12-31')
-0.7857190959433489
-1.1832868183769858
('TSLA', '2011-09-30')
-0.7555451702196498
nan
('TSLA', '2011-06-30')
-1.6427075026693878
nan
('TTWO', '2019-12-31')
0.07717224810583612
0.09989110983785957
('TTWO', '2019-09-30')
0.0970871820855228
0.057638283942668234
('TTWO', '2019-06-30')
0.08924295226510154
0.0574455595001817
('TTWO', '2019-03-31')
0.08931278778310794
0.05510464080194236
('TTWO', '2018-12-31')
0.09989110983785957
0.058903906932086025
('TTWO', '2018-09-30')
0.057638283942668234
0.042623225962377644
('TTWO', '2018-06-30')
0.0574455595001817
0.06544461062571912
('TTWO', '2018-03-31')
0.05510464080194236
0.025982935415787577
('TTWO', '2017-12-31')
0.058903906932086025
0.005579090827149925
('TTWO', '2017-09-30')
0.04262322596

('VRSK', '2016-09-30')
0.10601507679970643
0.1834372782611943
('VRSK', '2016-06-30')
0.10387482787690348
0.17134337864826707
('VRSK', '2016-03-31')
0.2085798330546197
0.14513993160616423
('VRSK', '2015-12-31')
0.21642029053480746
0.15973241241294
('VRSK', '2015-09-30')
0.1834372782611943
0.1604590694489257
('VRSK', '2015-06-30')
0.17134337864826707
0.15878034772363633
('VRSK', '2015-03-31')
0.14513993160616423
0.1523744539207033
('VRSK', '2014-12-31')
0.15973241241294
0.1475976301679083
('VRSK', '2014-09-30')
0.1604590694489257
0.15605229238271745
('VRSK', '2014-06-30')
0.15878034772363633
0.18664974589161315
('VRSK', '2014-03-31')
0.1523744539207033
0.1770847702721344
('VRSN', '2020-03-31')
0.40834390803448084
0.21021862285191698
('VRSN', '2019-12-31')
0.3198212174014784
0.19804548366170405
('VRSN', '2019-09-30')
0.3428628885912764
0.1729945677344953
('VRSN', '2019-06-30')
0.3295359609957182
0.20490553272665768
('VRSN', '2019-03-31')
0.21021862285191698
0.20518035049593633
('VRSN', '2

('WDC', '2020-04-03')
-0.02210827481142942
0.006861122603778789
('WDC', '2020-01-03')
-0.04270016822363005
0.028183646112600536
('WDC', '2019-10-04')
-0.05368402717296638
0.016554663169972134
('WDC', '2019-06-28')
-0.025791003933641185
0.02260549229738781
('WDC', '2019-03-29')
0.006861122603778789
0.006757674544960609
('WDC', '2018-12-28')
0.028183646112600536
0.013321829163071614
('WDC', '2018-09-28')
0.016554663169972134
0.04977593933126508
('WDC', '2018-06-29')
0.02260549229738781
0.012080822834885277
('WDC', '2018-03-30')
0.006757674544960609
-0.016658861309961866
('WDC', '2017-12-29')
0.013321829163071614
-0.027350316824001034
('WDC', '2017-09-29')
0.04977593933126508
-0.026669287726885524
('WDC', '2017-06-30')
0.012080822834885277
0.01594097885514788
('WDC', '2017-03-31')
-0.016658861309961866
0.05455983131259884
('WDC', '2016-12-30')
-0.027350316824001034
0.07517009049474867
('WDC', '2016-09-30')
-0.026669287726885524
0.0846537183746486
('WDC', '2016-07-01')
0.01594097885514788


In [15]:
# Question 4. Quality of Earnings (Accrual)
"""
Compare Cash flow return on assets (2) to return on assets (1)
Score 1 if CFROA > ROA, 0 if CFROA <= ROA.
"""
piotroski_dfs4 = pd.DataFrame(columns=['ticker','date','score4'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:4+item]
        
        # Calculate if no data is missing
        if((temp33['Net Income'].all()) and (len(temp33.index) == 4) and (temp33['Operating Cash Flow'].all())):
            
            # Calculating 1st score
            roa = temp33['Net Income'].sum()
            
            # Calculating 2st score
            cfroa = temp33['Operating Cash Flow'].sum()
            
            print(new_df.index[item])
            print(roa)
            print(cfroa)
            
            if(cfroa > roa):
                piotroski_dfs4 = piotroski_dfs4.append({'ticker':ticker, 'date':new_df.index[item][1], 'score4':1}, ignore_index=True)
            else:
                piotroski_dfs4 = piotroski_dfs4.append({'ticker':ticker, 'date':new_df.index[item][1], 'score4':0}, ignore_index=True)
        else:
            piotroski_dfs4 = piotroski_dfs4.append({'ticker':ticker, 'date':new_df.index[item][1], 'score4':None}, ignore_index=True)

('AAPL', '2020-03-28')
57215000000.0
75373000000.0
('AAPL', '2019-12-28')
57527000000.0
73217000000.0
('AAPL', '2019-09-28')
55256000000.0
69391000000.0
('AAPL', '2019-06-29')
55695000000.0
69004000000.0
('AAPL', '2019-03-30')
57170000000.0
71856000000.0
('AAPL', '2018-12-29')
59431000000.0
75831000000.0
('AAPL', '2018-09-29')
59531000000.0
77434000000.0
('AAPL', '2018-06-30')
56120000000.0
73567000000.0
('AAPL', '2018-03-31')
53318000000.0
67442000000.0
('AAPL', '2017-12-30')
50525000000.0
64835000000.0
('AAPL', '2017-09-30')
48351000000.0
63598000000.0
('AAPL', '2017-07-01')
46651000000.0
64068000000.0
('AAPL', '2017-04-01')
45730000000.0
66339000000.0
('AAPL', '2016-12-31')
45217000000.0
65417000000.0
('AAPL', '2016-09-24')
45687000000.0
65824000000.0
('AAPL', '2016-06-25')
47797000000.0
63173000000.0
('AAPL', '2016-03-26')
50678000000.0
67527000000.0
('AAPL', '2015-12-26')
53731000000.0
75007000000.0
('AAPL', '2015-09-26')
53394000000.0
81266000000.0
('AAPL', '2015-06-27')
50737000

('ALGN', '2014-12-31')
145832000.0
226899000.0
('ALGN', '2014-09-30')
148712000.0
222406000.0
('ALGN', '2014-06-30')
145002000.0
209973000.0
('ALGN', '2014-03-31')
138722000.0
193551000.0
('ALGN', '2013-12-31')
64295000.0
185976000.0
('ALGN', '2013-09-30')
31433000.0
169622000.0
('ALGN', '2013-06-30')
-3448000.0
154749000.0
('ALGN', '2013-03-31')
-4276000.0
128772000.0
('ALGN', '2012-12-31')
58691000.0
133778000.0
('ALGN', '2012-09-30')
69581000.0
125010000.0
('ALGN', '2012-06-30')
89189000.0
126280000.0
('ALGN', '2012-03-31')
71859000.0
128645000.0
('ALGN', '2011-12-31')
66716000.0
130469000.0
('ALGN', '2011-09-30')
56172000.0
121004000.0
('ALGN', '2011-06-30')
53723000.0
115232000.0
('ALGN', '2011-03-31')
90094000.0
146777000.0
('ALGN', '2010-12-31')
89183000.0
148174000.0
('ALGN', '2010-09-30')
90770000.0
149908000.0
('ALGN', '2010-06-30')
24013000.0
124938000.0
('ALGN', '2010-03-31')
-18975000.0
82207000.0
('ALXN', '2020-03-31')
2374000000.0
2204600000.0
('ALXN', '2019-12-31')
2404

('AMZN', '2010-12-31')
1152000000.0
3495000000.0
('AMZN', '2010-09-30')
1120000000.0
2618000000.0
('AMZN', '2010-06-30')
1088000000.0
2562000000.0
('AMZN', '2010-03-31')
1024000000.0
2780000000.0
('ANSS', '2020-03-31')
411129000.0
495770000.0
('ANSS', '2019-12-31')
451295000.0
499936000.0
('ANSS', '2019-09-30')
438606000.0
493419000.0
('ANSS', '2019-06-30')
438479000.0
483012000.0
('ANSS', '2019-03-31')
421325000.0
505594000.0
('ANSS', '2018-12-31')
419375000.0
486437000.0
('ANSS', '2018-09-30')
318797000.0
456981000.0
('ANSS', '2018-06-30')
303091000.0
435905000.0
('ANSS', '2018-03-31')
280225000.0
436966000.0
('ANSS', '2017-12-31')
259251000.0
430438000.0
('ANSS', '2017-09-30')
276649000.0
423194000.0
('ANSS', '2017-06-30')
272576000.0
416298000.0
('ANSS', '2017-03-31')
272474000.0
374145000.0
('ANSS', '2016-12-31')
265636000.0
356827000.0
('ANSS', '2016-09-30')
263674000.0
369789000.0
('ANSS', '2016-06-30')
260150000.0
365595000.0
('ANSS', '2016-03-31')
252857000.0
361978000.0
('ANS

('BKNG', '2011-06-30')
719867000.0
984783000.0
('BKNG', '2011-03-31')
620657000.0
1045854000.0
('BKNG', '2010-12-31')
569742000.0
874332000.0
('BKNG', '2010-09-30')
512468000.0
840680000.0
('BKNG', '2010-06-30')
608470000.0
730495000.0
('BKNG', '2010-03-31')
518323000.0
540198000.0
('BMRN', '2020-03-31')
98982000.0
30593000.0
('BMRN', '2019-12-31')
-38872000.0
-9081000.0
('BMRN', '2019-09-30')
-42522000.0
40293000.0
('BMRN', '2019-06-30')
-110178000.0
17368000.0
('BMRN', '2019-03-31')
-89535000.0
7849000.0
('BMRN', '2018-12-31')
-77211000.0
20208000.0
('BMRN', '2018-09-30')
-124953000.0
-14552000.0
('BMRN', '2018-06-30')
-124860000.0
-23460000.0
('BMRN', '2018-03-31')
-144901000.0
29177000.0
('BMRN', '2017-12-31')
-117042000.0
-8757000.0
('BMRN', '2017-09-30')
-144361000.0
-23018000.0
('BMRN', '2017-06-30')
-174631000.0
-71927000.0
('BMRN', '2017-03-31')
-561358000.0
-136727000.0
('BMRN', '2016-12-31')
-630210000.0
-227837000.0
('BMRN', '2016-09-30')
-482882000.0
-296227000.0
('BMRN', 

('CMCSA', '2019-03-31')
12166000000.0
26054000000.0
('CMCSA', '2018-12-31')
11731000000.0
24297000000.0
('CMCSA', '2018-09-30')
24205000000.0
23949000000.0
('CMCSA', '2018-06-30')
23969000000.0
23126000000.0
('CMCSA', '2018-03-31')
23266000000.0
21221000000.0
('CMCSA', '2017-12-31')
22714000000.0
21403000000.0
('CMCSA', '2017-09-30')
10025000000.0
21704000000.0
('CMCSA', '2017-06-30')
9612000000.0
20671000000.0
('CMCSA', '2017-03-31')
9127000000.0
19786000000.0
('CMCSA', '2016-12-31')
8695000000.0
19240000000.0
('CMCSA', '2016-09-30')
8370000000.0
18462000000.0
('CMCSA', '2016-06-30')
8129000000.0
19327000000.0
('CMCSA', '2016-03-31')
8238000000.0
18643000000.0
('CMCSA', '2015-12-31')
8163000000.0
18778000000.0
('CMCSA', '2015-09-30')
8117000000.0
18456000000.0
('CMCSA', '2015-06-30')
8713000000.0
18232000000.0
('CMCSA', '2015-03-31')
8568000000.0
17704000000.0
('CMCSA', '2014-12-31')
8380000000.0
16945000000.0
('CMCSA', '2014-09-30')
8368000000.0
14783000000.0
('CMCSA', '2014-06-30')


...

('REGN', '2016-03-31')
725771000.0
1447622000.0
('REGN', '2015-12-31')
636056000.0
1330780000.0
('REGN', '2015-09-30')
591238000.0
1247240000.0
('REGN', '2015-06-30')
460560000.0
517944000.0
('REGN', '2015-03-31')
358652000.0
602128000.0
('REGN', '2014-12-31')
348074000.0
743157000.0
('REGN', '2014-09-30')
334704000.0
821655000.0
('REGN', '2014-06-30')
396290000.0
750553000.0
('REGN', '2014-03-31')
390931000.0
550927000.0
('REGN', '2013-12-31')
424362000.0
583648000.0
('REGN', '2013-09-30')
797963000.0
400052000.0
('REGN', '2013-06-30')
848125000.0
305215000.0
('REGN', '2013-03-31')
837492000.0
98674000.0
('REGN', '2012-12-31')
750269000.0
-74615000.0
('REGN', '2012-09-30')
226419000.0
-218117000.0
('REGN', '2012-06-30')
-27414000.0
-262470000.0
('REGN', '2012-03-31')
-166662000.0
-218143000.0
('REGN', '2011-12-31')
-221760000.0
-141682000.0
('REGN', '2011-09-30')
-182914000.0
-133064000.0
('REGN', '2011-06-30')
-154424000.0
74340000.0
('REGN', '2011-03-31')
-147915000.0
85764000.0
('R

('SNPS', '2015-01-31')
256617000.0
537502000.0
('SNPS', '2014-10-31')
259124000.0
550953000.0
('SNPS', '2014-07-31')
253559000.0
569376000.0
('SNPS', '2014-04-30')
240200000.0
492324000.0
('SNPS', '2014-01-31')
245574000.0
514251000.0
('SNPS', '2013-10-31')
247800000.0
496705000.0
('SNPS', '2013-07-31')
219487000.0
408302000.0
('SNPS', '2013-04-30')
242846000.0
440904000.0
('SNPS', '2013-01-31')
195126000.0
433948000.0
('SNPS', '2012-10-31')
181898000.0
486068000.0
('SNPS', '2012-07-31')
193263000.0
456162000.0
('SNPS', '2012-04-30')
169689000.0
470917000.0
('SNPS', '2012-01-31')
229832000.0
440631000.0
('SNPS', '2011-10-31')
221364000.0
440316000.0
('SNPS', '2011-07-31')
206823000.0
464182000.0
('SNPS', '2011-04-30')
273227000.0
266806000.0
('SNPS', '2011-01-31')
231662000.0
251963000.0
('SNPS', '2010-10-31')
316222000.0
246179000.0
('SNPS', '2010-07-31')
310349000.0
213197000.0
('SNPS', '2010-04-30')
239299000.0
332602000.0
('SPLK', '2020-01-31')
-336668000.0
-287636000.0
('SPLK', '2

('TTWO', '2015-12-31')
-297492000.0
341111000.0
('TTWO', '2015-09-30')
-214986000.0
364650000.0
('TTWO', '2015-06-30')
-311090000.0
405812000.0
('TTWO', '2015-03-31')
-279470000.0
212814000.0
('TTWO', '2014-12-31')
-67466000.0
29995000.0
('TTWO', '2014-09-30')
470867000.0
433319000.0
('TTWO', '2014-06-30')
388116000.0
605545000.0
('TTWO', '2014-03-31')
361605000.0
700262000.0
('TTWO', '2013-12-31')
414865000.0
684764000.0
('TTWO', '2013-09-30')
-92196000.0
212872000.0
('TTWO', '2013-06-30')
19433000.0
55150000.0
('TTWO', '2013-03-31')
-29491000.0
-4567000.0
('TTWO', '2012-12-31')
-118800000.0
11103000.0
('TTWO', '2012-09-30')
-176061000.0
-157050000.0
('TTWO', '2012-06-30')
-210941000.0
-109074000.0
('TTWO', '2012-03-31')
-108816000.0
-84964000.0
('TTWO', '2011-12-31')
-64034000.0
-74978000.0
('TTWO', '2011-09-30')
3570000.0
28732000.0
('TTWO', '2011-06-30')
54280000.0
38672000.0
('TTWO', '2011-03-31')
56260000.0
124349000.0
('TTWO', '2010-12-31')
104628000.0
181855000.0
('TTWO', '2010

('VRSN', '2016-09-30')
436623000.0
661012000.0
('VRSN', '2016-06-30')
414653000.0
648148000.0
('VRSN', '2016-03-31')
394454000.0
662439000.0
('VRSN', '2015-12-31')
375236000.0
651482000.0
('VRSN', '2015-09-30')
339178000.0
633420000.0
('VRSN', '2015-06-30')
341910000.0
645836000.0
('VRSN', '2015-03-31')
349075000.0
591989000.0
('VRSN', '2014-12-31')
355260000.0
600949000.0
('VRSN', '2014-09-30')
581937000.0
577916000.0
('VRSN', '2014-06-30')
567646000.0
544662000.0
('VRSN', '2014-03-31')
554360000.0
570389000.0
('VRSN', '2013-12-31')
544450000.0
579397000.0
('VRSN', '2013-09-30')
357942000.0
602716000.0
('VRSN', '2013-06-30')
354954000.0
589854000.0
('VRSN', '2013-03-31')
336536000.0
577989000.0
('VRSN', '2012-12-31')
320032000.0
537630000.0
('VRSN', '2012-09-30')
268205000.0
491085000.0
('VRSN', '2012-06-30')
249211000.0
477914000.0
('VRSN', '2012-03-31')
170129000.0
355892000.0
('VRSN', '2011-12-31')
142891000.0
335901000.0
('VRSN', '2011-09-30')
48572000.0
258414000.0
('VRSN', '2011

('XEL', '2014-12-31')
1021306000.0
2648192000.0
('XEL', '2014-09-30')
975022000.0
2585469000.0
('XEL', '2014-06-30')
971192000.0
2549322000.0
('XEL', '2014-03-31')
972885000.0
2534357000.0
('XEL', '2013-12-31')
948234000.0
2584036000.0
('XEL', '2013-09-30')
938349000.0
2435743000.0
('XEL', '2013-06-30')
971703000.0
2213685000.0
('XEL', '2013-03-31')
957906000.0
2169601000.0
('XEL', '2012-12-31')
905229000.0
2004756000.0
('XEL', '2012-09-30')
905568000.0
2053325000.0
('XEL', '2012-06-30')
845794000.0
2147702000.0
('XEL', '2012-03-31')
822556000.0
2223562000.0
('XEL', '2011-12-31')
841172000.0
2405522000.0
('XEL', '2011-09-30')
1147482000.0
2906529000.0
('XEL', '2011-06-30')
1121456000.0
2718635000.0
('XEL', '2011-03-31')
1100350000.0
2610954000.0
('XEL', '2010-12-31')
1063899000.0
2507050000.0
('XEL', '2010-09-30')
785965000.0
1808729000.0
('XEL', '2010-06-30')
694487000.0
1698671000.0
('XEL', '2010-03-31')
672878000.0
1655897000.0
('XLNX', '2020-03-28')
630463000.0
845485000.0
('XLNX',

In [16]:
# Question 5. Change in Gearing or Leverage
"""
Compare this year's gearing (long-term debt divided by average total assets) to last year's gearing.
Score 0 if this year's gearing is higher, 1 otherwise.
"""
piotroski_dfs5 = pd.DataFrame(columns=['ticker','date','score5'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:9+item]
        
        # Calculate if no data is missing
        if((temp33['Total assets'].all()) and (len(temp33.index) == 9) 
           and (temp33['Long-term debt'][0] != 0) and (temp33['Long-term debt'][4] != 0)):
            
            # Calculating 5th score
            gearing_this_year = temp33['Long-term debt'][0] / temp33['Total assets'][0:5].mean()
            gearing_last_year = temp33['Long-term debt'][4] / temp33['Total assets'][4:9].mean()
            
            print(new_df.index[item])
            print(gearing_this_year)
            print(gearing_last_year)
            
            if(gearing_this_year < gearing_last_year):
                piotroski_dfs5 = piotroski_dfs5.append({'ticker':ticker, 'date':new_df.index[item][1], 'score5':1}, ignore_index=True)
            else:
                piotroski_dfs5 = piotroski_dfs5.append({'ticker':ticker, 'date':new_df.index[item][1], 'score5':0}, ignore_index=True)
        else:
            piotroski_dfs5 = piotroski_dfs5.append({'ticker':ticker, 'date':new_df.index[item][1], 'score5':None}, ignore_index=True)

('AAPL', '2020-03-28')
0.2677231421872361
0.250817371941355
('AAPL', '2019-12-28')
0.2710341333302273
0.24957634101421572
('AAPL', '2019-09-28')
0.26348053635725466
0.25136267073273416
('AAPL', '2019-06-29')
0.24227584578048217
0.26336439830042
('AAPL', '2019-03-30')
0.250817371941355
0.27704830210132725
('AAPL', '2018-12-29')
0.24957634101421572
0.2898058460901783
('AAPL', '2018-09-29')
0.25136267073273416
0.28458864385710464
('AAPL', '2018-06-30')
0.26336439830042
0.2742876956341789
('AAPL', '2018-03-31')
0.27704830210132725
0.26445060122459857
('AAPL', '2017-12-30')
0.2898058460901783
0.23621538995112365
('AAPL', '2017-09-30')
0.28458864385710464
0.24871597701816495
('AAPL', '2017-07-01')
0.2742876956341789
0.2348389725254174
('AAPL', '2017-04-01')
0.26445060122459857
0.24369373008708115
('AAPL', '2016-12-31')
0.23621538995112365
0.1927678365683528
('AAPL', '2016-09-24')
0.24871597701816495
0.20273298765241093
('AAPL', '2016-06-25')
0.2348389725254174
0.18958530239133597
('AAPL', '2

('ALXN', '2020-03-31')
0.14816870986425615
0.17873760595811317
('ALXN', '2019-12-31')
0.15863198448278626
0.18212884795149636
('ALXN', '2019-09-30')
0.16911006242463933
0.18520847193689183
('ALXN', '2019-06-30')
0.17458028559481736
0.18849607339654506
('ALXN', '2019-03-31')
0.17873760595811317
0.19702708865042182
('ALXN', '2018-12-31')
0.18212884795149636
0.20182246269841386
('ALXN', '2018-09-30')
0.18520847193689183
0.20584543828181595
('ALXN', '2018-06-30')
0.18849607339654506
0.21082767320429446
('ALXN', '2018-03-31')
0.19702708865042182
0.21621784572451014
('ALXN', '2017-12-31')
0.20182246269841386
0.2204193695261124
('ALXN', '2017-09-30')
0.20584543828181595
0.22391965058009142
('ALXN', '2017-06-30')
0.21082767320429446
0.24286247064367303
('ALXN', '2017-03-31')
0.21621784572451014
0.2834565355387611
('ALXN', '2016-12-31')
0.2204193695261124
0.34129310504613286
('ALXN', '2016-09-30')
0.22391965058009142
0.42816479067390384
('ALXN', '2016-06-30')
0.24286247064367303
0.5762624582597

('ATVI', '2017-12-31')
0.2506652049287974
0.2874976468373494
('ATVI', '2017-09-30')
0.251377765556434
0.2981965250116077
('ATVI', '2017-06-30')
0.254560857859066
0.3211920957187294
('ATVI', '2017-03-31')
0.2534559553206711
0.3869545996490147
('ATVI', '2016-12-31')
0.2874976468373494
0.2829063267259436
('ATVI', '2016-09-30')
0.2981965250116077
0.2884220949147747
('ATVI', '2016-06-30')
0.3211920957187294
0.29224549481742723
('ATVI', '2016-03-31')
0.3869545996490147
0.29415586290532153
('ATVI', '2015-12-31')
0.2829063267259436
0.3110746607962475
('ATVI', '2015-09-30')
0.2884220949147747
0.3051613358751677
('ATVI', '2015-06-30')
0.29224549481742723
0.3070156740702846
('ATVI', '2015-03-31')
0.29415586290532153
0.30536077810450124
('ATVI', '2014-12-31')
0.3110746607962475
0.32685413399058927
('ATVI', '2014-09-30')
0.3051613358751677
0.15780007708009192
('AVGO', '2020-02-02')
0.5868798644863434
0.6044965010670504
('AVGO', '2019-11-03')
0.4541478768802397
0.3308131046846751
('AVGO', '2019-08-0

('CDNS', '2017-07-01')
0.2893334825097403
0.26840101655257786
('CDNS', '2017-04-01')
0.282351047320444
0.25380679395947553
('CDNS', '2016-12-31')
0.27835374053965484
0.13077984892158032
('CDNS', '2016-10-01')
0.2727335783967271
0.12737899707484118
('CDNS', '2016-07-02')
0.26840101655257786
0.12327470626845057
('CDNS', '2016-04-02')
0.25380679395947553
0.12202624348978154
('CDNS', '2016-01-02')
0.13077984892158032
0.1276813183038623
('CDW', '2020-03-31')
0.43496355591087454
0.44527700475115223
('CDW', '2019-12-31')
0.4267043741357261
0.4438795572499073
('CDW', '2019-09-30')
0.4305424914657733
0.44526958688861695
('CDW', '2019-06-30')
0.43406734161271904
0.45147780472671895
('CDW', '2019-03-31')
0.44527700475115223
0.45980815182823054
('CDW', '2018-12-31')
0.4438795572499073
0.46090095367730044
('CDW', '2018-09-30')
0.44526958688861695
0.48909764342812934
('CDW', '2018-06-30')
0.45147780472671895
0.4760728996612966
('CDW', '2018-03-31')
0.45980815182823054
0.47827527659911384
('CDW', '20

('COST', '2020-02-16')
0.10981461380748092
0.11650004131207138
('COST', '2019-11-24')
0.1123919435201324
0.16013443384569762
('COST', '2019-09-01')
0.11828527897689235
0.16645027532163623
('COST', '2019-05-12')
0.11382827324478179
0.17114476126202127
('COST', '2019-02-17')
0.11650004131207138
0.17515846388315948
('COST', '2018-11-25')
0.16013443384569762
0.17649494872437582
('COST', '2018-09-02')
0.16645027532163623
0.18536065379604405
('COST', '2018-05-13')
0.17114476126202127
0.08067883474709574
('COST', '2018-02-18')
0.17515846388315948
0.08202883685149139
('COST', '2017-11-26')
0.17649494872437582
0.1147269364728395
('COST', '2017-09-03')
0.18536065379604405
0.12063690104862906
('COST', '2017-05-07')
0.08067883474709574
0.11625148983355373
('COST', '2017-02-12')
0.08202883685149139
0.14509710756072935
('COST', '2016-11-20')
0.1147269364728395
0.14200298953662183
('COST', '2016-08-28')
0.12063690104862906
0.14461728747443034
('COST', '2016-05-08')
0.11625148983355373
0.1441207914996

...

('NFLX', '2011-06-30')
0.03229360571725855
0.05663813055696764
('NTAP', '2019-10-25')
0.13965799038860294
0.12063438501771553
('NTAP', '2019-07-26')
0.13494083816527602
0.15908715747771543
('NTAP', '2019-04-26')
0.12859712230215828
0.15707820272364023
('NTAP', '2019-01-25')
0.125642490005711
0.15961195638655115
('NTAP', '2018-10-26')
0.12063438501771553
0.1630768579113455
('NTAP', '2018-07-27')
0.15908715747771543
0.08151698179271709
('NTAP', '2018-04-27')
0.15707820272364023
0.08063467290935102
('NTAP', '2018-01-26')
0.15961195638655115
0.08189142781666887
('NTAP', '2017-10-27')
0.1630768579113455
0.16575567702084165
('NTAP', '2017-07-28')
0.08151698179271709
0.16588415923099176
('NTAP', '2017-04-28')
0.08063467290935102
0.16353786175892104
('NTAP', '2017-01-27')
0.08189142781666887
0.1664696589494759
('NTAP', '2016-10-28')
0.16575567702084165
0.16491159675182854
('NTAP', '2016-07-29')
0.16588415923099176
0.16179962942665518
('NTAP', '2016-04-29')
0.16353786175892104
0.159929039888184

('PCAR', '2012-03-31')
0.009023153411654305
0.010598669372389194
('PCAR', '2011-12-31')
0.009470199177229096
0.010683928095739393
('PCAR', '2011-09-30')
0.009867564132588486
0.010598879062550345
('PCAR', '2011-06-30')
0.01026348416549663
0.012001193843338745
('PEP', '2020-03-21')
0.4514904124840719
0.3673289206018112
('PEP', '2019-12-28')
0.3793445413585501
0.3611801828941389
('PEP', '2019-09-07')
0.39061470071768695
0.3903397814616947
('PEP', '2019-06-15')
0.36409406113605963
0.38701153026061824
('PEP', '2019-03-23')
0.3673289206018112
0.4075753282327383
('PEP', '2018-12-29')
0.3611801828941389
0.44009678066261937
('PEP', '2018-09-08')
0.3903397814616947
0.4159723214144011
('PEP', '2018-06-16')
0.38701153026061824
0.4190818206724648
('PEP', '2018-03-24')
0.4075753282327383
0.41164218973227723
('PEP', '2017-12-30')
0.44009678066261937
0.41691290511426854
('PEP', '2017-09-09')
0.4159723214144011
0.4115992318835697
('PEP', '2017-06-17')
0.4190818206724648
0.4350027216755368
('PEP', '2017

('SIRI', '2020-03-31')
0.7042342002764166
0.8110516259602931
('SIRI', '2019-12-31')
0.740499622139341
0.8319870187259123
('SIRI', '2019-09-30')
0.7892123459091351
0.7839385628127513
('SIRI', '2019-06-30')
0.8293082680839481
0.7683766021776031
('SIRI', '2019-03-31')
0.8110516259602931
0.8189787882693094
('SIRI', '2018-12-31')
0.8319870187259123
0.8168254154045467
('SIRI', '2018-09-30')
0.7839385628127513
0.8134209083979853
('SIRI', '2018-06-30')
0.7683766021776031
0.7900152076920951
('SIRI', '2018-03-31')
0.8189787882693094
0.7400146414532217
('SIRI', '2017-12-31')
0.8168254154045467
0.7205963537659038
('SIRI', '2017-09-30')
0.8134209083979853
0.7059497119077058
('SIRI', '2017-06-30')
0.7900152076920951
0.7496622110220612
('SIRI', '2017-03-31')
0.7400146414532217
0.6950063014522738
('SIRI', '2016-12-31')
0.7205963537659038
0.6555233128037068
('SIRI', '2016-09-30')
0.7059497119077058
0.6440634582334289
('SIRI', '2016-06-30')
0.7496622110220612
0.6022918598322748
('SIRI', '2016-03-31')
0.

('TTWO', '2012-09-30')
0.29812241097281394
0.11463041786149523
('TTWO', '2012-06-30')
0.302796767404848
0.11215212343756176
('TTWO', '2012-03-31')
0.3043384146194808
0.1112277379091604
('TTWO', '2011-12-31')
0.3103945647066176
0.11446054650051411
('TTWO', '2011-09-30')
0.11463041786149523
nan
('TTWO', '2011-06-30')
0.11215212343756176
0.10988857803962856
('TTWO', '2011-03-31')
0.1112277379091604
nan
('TXN', '2020-03-31')
0.3120177031320926
0.2835054436184646
('TXN', '2019-12-31')
0.3013958669606929
0.24159264314322154
('TXN', '2019-09-30')
0.2994498977736109
0.2421000695238736
('TXN', '2019-06-30')
0.2558834096829246
0.29100559493583633
('TXN', '2019-03-31')
0.2835054436184646
0.21173101048595167
('TXN', '2018-12-31')
0.24159264314322154
0.2144021673979237
('TXN', '2018-09-30')
0.2421000695238736
0.1880671285003415
('TXN', '2018-06-30')
0.29100559493583633
0.19094556441626628
('TXN', '2018-03-31')
0.21173101048595167
0.18624068796560173
('TXN', '2017-12-31')
0.2144021673979237
0.185531

('VRSN', '2013-03-31')
0.050293528118155996
0.05250527888073867
('VRSN', '2012-12-31')
0.051404879947501224
0.04958301674577141
('VRSN', '2012-09-30')
0.053636926921618
0.2629868845958676
('VRSN', '2012-06-30')
0.054738893834243844
0.2460677907267191
('VRSN', '2012-03-31')
0.05250527888073867
0.232003194190439
('VRSN', '2011-12-31')
0.04958301674577141
0.22858371288087886
('VRSN', '2011-09-30')
0.2629868845958676
0.22413530336263032
('VRSN', '2011-06-30')
0.2460677907267191
0.2298962480479498
('VRTX', '2020-03-31')
0.06957088572418875
0.012351832104116816
('VRTX', '2019-12-31')
0.07552495195622129
0.0043297438469093496
('VRTX', '2019-09-30')
0.009853283115561247
0.003851978007624988
('VRTX', '2019-06-30')
0.010499602327540127
0.004102456489805027
('VRTX', '2019-03-31')
0.012351832104116816
0.00499450371272581
('VRTX', '2018-12-31')
0.0043297438469093496
0.006456074408147505
('VRTX', '2018-09-30')
0.003851978007624988
0.006781459525140507
('VRTX', '2018-06-30')
0.004102456489805027
0.00

In [17]:
# Question 6. Change in Working Capital (Liquidity)
"""
Compare this year's current ratio (current assets divided by current liabilities) to last year's current ratio.
Score 1 if this year's current ratio is higher, 0 if it's lower
"""
piotroski_dfs6 = pd.DataFrame(columns=['ticker','date','score6'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:5+item]
        
        # Calculate if no data is missing
        if((len(temp33.index) == 5) and (temp33['Total current assets'][0] != 0) and (temp33['Total current assets'][4] != 0) 
           and (temp33['Total current liabilities'][0] != 0) and (temp33['Total current liabilities'][4] != 0)):
            
            # Calculating 6th score
            cr_this_year = temp33['Total current assets'][0] / temp33['Total current liabilities'][0]
            cr_last_year = temp33['Total current assets'][4] / temp33['Total current liabilities'][4]
            
            print(new_df.index[item])
            print(cr_this_year)
            print(cr_last_year)
            
            if(cr_this_year > cr_last_year):
                piotroski_dfs6 = piotroski_dfs6.append({'ticker':ticker, 'date':new_df.index[item][1], 'score6':1}, ignore_index=True)
            else:
                piotroski_dfs6 = piotroski_dfs6.append({'ticker':ticker, 'date':new_df.index[item][1], 'score6':0}, ignore_index=True)
        else:
            piotroski_dfs6 = piotroski_dfs6.append({'ticker':ticker, 'date':new_df.index[item][1], 'score6':None}, ignore_index=True)

('AAPL', '2020-03-28')
1.4959622869273836
1.3153819903595956
('AAPL', '2019-12-28')
1.5977819324399722
1.3005550271049011
('AAPL', '2019-09-28')
1.540125617208044
1.1238426916297297
('AAPL', '2019-06-29')
1.504648622135022
1.3073248407643312
('AAPL', '2019-03-30')
1.3153819903595956
1.4560344827586207
('AAPL', '2018-12-29')
1.3005550271049011
1.2420112619615158
('AAPL', '2018-09-29')
1.1238426916297297
1.2760628484139107
('AAPL', '2018-06-30')
1.3073248407643312
1.388342230203439
('AAPL', '2018-03-31')
1.4560344827586207
1.3906083826456874
('AAPL', '2017-12-30')
1.2420112619615158
1.228242006418638
('AAPL', '2017-09-30')
1.2760628484139107
1.352669417512594
('AAPL', '2017-07-01')
1.388342230203439
1.3115994740228856
('AAPL', '2017-04-01')
1.3906083826456874
1.2831172636050685
('AAPL', '2016-12-31')
1.228242006418638
1.0016690322241497
('AAPL', '2016-09-24')
1.352669417512594
1.1087706239920605
('AAPL', '2016-06-25')
1.3115994740228856
1.0868193306272498
('AAPL', '2016-03-26')
1.2831172

('ADSK', '2012-07-31')
2.110827601508764
2.1506468383508643
('ADSK', '2012-04-30')
2.1840091376356368
2.1139719341061625
('ADSK', '2012-01-31')
1.98710826957342
1.970912853529547
('ADSK', '2011-10-31')
2.087758112094395
2.0019055396760583
('ADSK', '2011-07-31')
2.1506468383508643
1.960171483888812
('ADSK', '2011-04-30')
2.1139719341061625
1.960618846694796
('ADSK', '2011-01-31')
1.970912853529547
1.959659090909091
('ADSK', '2010-10-31')
2.0019055396760583
1.6392475689462778
('ADSK', '2010-07-31')
1.960171483888812
1.577452017530603
('ALGN', '2020-03-31')
1.7311400416828633
1.7879756586042213
('ALGN', '2019-12-31')
1.6822548585435184
1.8819965523868147
('ALGN', '2019-09-30')
1.7302641861576935
1.9802073408830994
('ALGN', '2019-06-30')
1.847007652278828
2.2253735136222614
('ALGN', '2019-03-31')
1.7879756586042213
2.256389663793353
('ALGN', '2018-12-31')
1.8819965523868147
2.324157921802433
('ALGN', '2018-09-30')
1.9802073408830994
2.3863827001814166
('ALGN', '2018-06-30')
2.2253735136222

('AMGN', '2018-03-31')
3.8829086744918406
4.414710633849663
('AMGN', '2017-12-31')
5.485144124168515
4.106569082470546
('AMGN', '2017-09-30')
6.072500915415599
4.348700436349839
('AMGN', '2017-06-30')
6.198592450415867
3.9655586334256694
('AMGN', '2017-03-31')
4.414710633849663
4.952012202276194
('AMGN', '2016-12-31')
4.106569082470546
4.444213684089074
('AMGN', '2016-09-30')
4.348700436349839
5.421353803210049
('AMGN', '2016-06-30')
3.9655586334256694
5.4761282832680305
('AMGN', '2016-03-31')
4.952012202276194
5.534090909090909
('AMGN', '2015-12-31')
4.444213684089074
4.953339041095891
('AMGN', '2015-09-30')
5.421353803210049
4.215647292714302
('AMGN', '2015-06-30')
5.4761282832680305
4.364098652428172
('AMGN', '2015-03-31')
5.534090909090909
3.4502011459222235
('AMGN', '2014-12-31')
4.953339041095891
3.4436894425569395
('AMGN', '2014-09-30')
4.215647292714302
6.33388395622548
('AMGN', '2014-06-30')
4.364098652428172
6.20255016722408
('AMGN', '2014-03-31')
3.4502011459222235
5.5805134

('AVGO', '2017-07-30')
4.482867448151488
2.2170087976539588
('AVGO', '2017-04-30')
4.1267056530214425
2.5224443606186346
('AVGO', '2017-01-29')
3.463713477851084
4.606728538283063
('AVGO', '2016-10-30')
2.314814814814815
3.359249329758713
('AVGO', '2016-07-31')
2.2170087976539588
3.328781512605042
('AVGO', '2016-05-01')
2.5224443606186346
4.498896247240618
('AVGO', '2016-01-31')
4.606728538283063
4.849704142011834
('AVGO', '2015-11-01')
3.359249329758713
3.7746062992125986
('AVGO', '2015-08-02')
3.328781512605042
4.085106382978723
('AVGO', '2015-05-03')
4.498896247240618
4.889423076923077
('AVGO', '2015-02-01')
4.849704142011834
5.107142857142857
('AVGO', '2014-11-02')
3.7746062992125986
4.391304347826087
('AVGO', '2014-08-03')
4.085106382978723
4.318421052631579
('AVGO', '2014-05-04')
4.889423076923077
5.509090909090909
('AVGO', '2014-02-02')
5.107142857142857
6.184115523465704
('AVGO', '2013-11-03')
4.391304347826087
4.88728323699422
('AVGO', '2013-08-04')
4.318421052631579
4.6228070

('CDNS', '2017-07-01')
1.6218275276515672
1.704567639435861
('CDNS', '2017-04-01')
1.389315522397391
2.0289558020778307
('CDNS', '2016-12-31')
1.1989063497972887
1.7988969004693207
('CDNS', '2016-10-01')
1.4911701212637936
2.044717225844555
('CDNS', '2016-07-02')
1.704567639435861
2.0237066309164655
('CDNS', '2016-04-02')
2.0289558020778307
1.4135625503491418
('CDNS', '2016-01-02')
1.7988969004693207
1.32922759532579
('CDNS', '2015-10-03')
2.044717225844555
1.0428735444167019
('CDNS', '2015-07-04')
2.0237066309164655
1.0047384781900035
('CDNS', '2015-04-04')
1.4135625503491418
1.0880619840462014
('CDNS', '2015-01-03')
1.32922759532579
1.063473580616786
('CDNS', '2014-09-27')
1.0428735444167019
0.9818700922408279
('CDNS', '2014-06-28')
1.0047384781900035
0.929213754577235
('CDNS', '2014-03-29')
1.0880619840462014
1.207567991739804
('CDNS', '2013-12-28')
1.063473580616786
1.1429140617940146
('CDNS', '2013-09-28')
0.9818700922408279
1.1637735455761955
('CDW', '2020-03-31')
1.2792827374424

...

('SIRI', '2019-12-31')
0.30458548436076527
0.17063430047975545
('SIRI', '2019-09-30')
0.28791141187326974
0.17568660892738175
('SIRI', '2019-06-30')
0.3314950980392157
0.1635262346648928
('SIRI', '2019-03-31')
0.28290704928259514
0.18087496332822217
('SIRI', '2018-12-31')
0.17063430047975545
0.16689514725656715
('SIRI', '2018-09-30')
0.17568660892738175
0.1850879141686956
('SIRI', '2018-06-30')
0.1635262346648928
0.18729705014057574
('SIRI', '2018-03-31')
0.18087496332822217
0.2622487904022342
('SIRI', '2017-12-31')
0.16689514725656715
0.24081869114191293
('SIRI', '2017-09-30')
0.1850879141686956
0.3588492361046021
('SIRI', '2017-06-30')
0.18729705014057574
0.3676408957203875
('SIRI', '2017-03-31')
0.2622487904022342
0.22446094124956378
('SIRI', '2016-12-31')
0.24081869114191293
0.22507187099311954
('SIRI', '2016-09-30')
0.3588492361046021
0.5847273778309969
('SIRI', '2016-06-30')
0.3676408957203875
0.6291749701691367
('SIRI', '2016-03-31')
0.22446094124956378
0.7848982250688733
('SIRI

('TMUS', '2017-09-30')
0.9192891521658645
1.6664613868703042
('TMUS', '2017-06-30')
0.7680108744902583
1.6316270566727604
('TMUS', '2017-03-31')
1.0401701014066078
1.4242715337816203
('TMUS', '2016-12-31')
1.5758146752383064
1.5627623845507976
('TMUS', '2016-09-30')
1.6664613868703042
1.4311885106864186
('TMUS', '2016-06-30')
1.6316270566727604
1.4105759922555663
('TMUS', '2016-03-31')
1.4242715337816203
1.3452422245346283
('TMUS', '2015-12-31')
1.5627623845507976
1.593436645396536
('TMUS', '2015-09-30')
1.4311885106864186
1.5450319581638583
('TMUS', '2015-06-30')
1.4105759922555663
1.3877436544664143
('TMUS', '2015-03-31')
1.3452422245346283
2.1206219312602292
('TMUS', '2014-12-31')
1.593436645396536
2.105371900826446
('TMUS', '2014-09-30')
1.5450319581638583
1.444323540079797
('TMUS', '2014-06-30')
1.3877436544664143
1.3547610226009634
('TMUS', '2014-03-31')
2.1206219312602292
2.1236872182808764
('TMUS', '2013-12-31')
2.105371900826446
3.759485399320361
('TMUS', '2013-09-30')
1.44432

('UAL', '2019-12-31')
0.5485339402865176
0.5445049954586739
('UAL', '2019-09-30')
0.5438149257536167
0.6336597486115171
('UAL', '2019-06-30')
0.5541734046523199
0.6302157252341755
('UAL', '2019-03-31')
0.49611998695793935
0.5842910526686879
('UAL', '2018-12-31')
0.5445049954586739
0.5611391606184917
('UAL', '2018-09-30')
0.6336597486115171
0.6018950103155803
('UAL', '2018-06-30')
0.6302157252341755
0.6066490144160047
('UAL', '2018-03-31')
0.5842910526686879
0.6100142157637024
('UAL', '2017-12-31')
0.5611391606184917
0.5949047696565196
('UAL', '2017-09-30')
0.6018950103155803
0.6013732739757036
('UAL', '2017-06-30')
0.6066490144160047
0.5610070090115863
('UAL', '2017-03-31')
0.6100142157637024
0.5399103139013453
('UAL', '2016-12-31')
0.5949047696565196
0.6305783792492348
('UAL', '2016-09-30')
0.6013732739757036
0.748591340450771
('UAL', '2016-06-30')
0.5610070090115863
0.6367125271542361
('UAL', '2016-03-31')
0.5399103139013453
0.6807215978237526
('UAL', '2015-12-31')
0.6305783792492348

('VRSN', '2010-12-31')
3.336775887717879
1.9279279482534175
('VRSN', '2010-09-30')
4.343077959186342
0.7600551657594731
('VRSN', '2010-06-30')
2.248769152120283
0.7666952448663795
('VRTX', '2020-03-31')
3.539496344435418
3.7805331921031606
('VRTX', '2019-12-31')
3.6130742036233907
3.4304529533371655
('VRTX', '2019-09-30')
3.4394950226583165
3.714439581343425
('VRTX', '2019-06-30')
3.7411794603008213
3.674563787339588
('VRTX', '2019-03-31')
3.7805331921031606
3.675899781460924
('VRTX', '2018-12-31')
3.4304529533371655
3.2814248197606717
('VRTX', '2018-09-30')
3.714439581343425
3.275427407247949
('VRTX', '2018-06-30')
3.674563787339588
3.4866502719833194
('VRTX', '2018-03-31')
3.675899781460924
3.821619613063398
('VRTX', '2017-12-31')
3.2814248197606717
2.310983588147935
('VRTX', '2017-09-30')
3.275427407247949
2.0777750570319062
('VRTX', '2017-06-30')
3.4866502719833194
2.334652527474292
('VRTX', '2017-03-31')
3.821619613063398
2.541179006631754
('VRTX', '2016-12-31')
2.310983588147935


('XEL', '2014-09-30')
0.9209780029151201
1.0992159027227664
('XEL', '2014-06-30')
1.0208798962328864
1.045012440624293
('XEL', '2014-03-31')
0.9024144776379368
1.0705136755462163
('XEL', '2013-12-31')
0.8805696432177552
0.8937942638810816
('XEL', '2013-09-30')
1.0992159027227664
1.0665425526192727
('XEL', '2013-06-30')
1.045012440624293
0.6683006100925502
('XEL', '2013-03-31')
1.0705136755462163
0.685761919633057
('XEL', '2012-12-31')
0.8937942638810816
0.8311128536473019
('XEL', '2012-09-30')
1.0665425526192727
1.0785057042804813
('XEL', '2012-06-30')
0.6683006100925502
0.9394427456239846
('XEL', '2012-03-31')
0.685761919633057
1.0118212814457435
('XEL', '2011-12-31')
0.8311128536473019
1.0773141922458727
('XEL', '2011-09-30')
1.0785057042804813
1.150048374982496
('XEL', '2011-06-30')
0.9394427456239846
0.96516129527337
('XEL', '2011-03-31')
1.0118212814457435
0.8494317420080454
('XEL', '2010-12-31')
1.0773141922458727
0.9633126094862814
('XEL', '2010-09-30')
1.150048374982496
9.50782

In [20]:
# Question 7. Change in Shares in Issue
"""
Compare the number of shares in issue this year, to the number in issue last year.
Score 0 if there is larger number of shares in issue this year, 1 otherwise.
"""
piotroski_dfs7 = pd.DataFrame(columns=['ticker','date','score7'])
for ticker, new_df in file4.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:5+item]
        
        # Calculate if no data is missing
        if((len(temp33.index) == 5) and (temp33['Number of Shares'][0] != 0) and (temp33['Number of Shares'][4] != 0)):
            
            print(new_df.index[item])
            print(temp33['Number of Shares'][0])
            print(temp33['Number of Shares'][4])
            
            if(temp33['Number of Shares'][0] < temp33['Number of Shares'][4]):
                piotroski_dfs7 = piotroski_dfs7.append({'ticker':ticker, 'date':new_df.index[item][1], 'score7':1}, ignore_index=True)
            else:
                piotroski_dfs7 = piotroski_dfs7.append({'ticker':ticker, 'date':new_df.index[item][1], 'score7':0}, ignore_index=True)
        else:
            piotroski_dfs7 = piotroski_dfs7.append({'ticker':ticker, 'date':new_df.index[item][1], 'score7':None}, ignore_index=True)

('AAPL', '2020-03-28')
4387570000.0
4704945000.0
('AAPL', '2019-12-28')
4415040000.0
4735820000.0
('AAPL', '2019-09-28')
4617834000.0
4955377000.0
('AAPL', '2019-06-29')
4660175000.0
5006640000.0
('AAPL', '2019-03-30')
4704945000.0
5068877000.0
('AAPL', '2018-12-29')
4735820000.0
5112877000.0
('AAPL', '2018-09-29')
4955377000.0
5217242000.0
('AAPL', '2018-06-30')
5006640000.0
5239847000.0
('AAPL', '2018-03-31')
5068877000.0
5262226000.0
('AAPL', '2017-12-30')
5112877000.0
5298661000.0
('AAPL', '2017-09-30')
5217242000.0
5470820000.0
('AAPL', '2017-07-01')
5239847000.0
5505456000.0
('AAPL', '2017-04-01')
5262226000.0
5536656000.0
('AAPL', '2016-12-31')
5298661000.0
5558930000.0
('AAPL', '2016-09-24')
5470820000.0
5753421000.0
('AAPL', '2016-06-25')
5505456000.0
5788922000.0
('AAPL', '2016-03-26')
5536656000.0
5818441000.0
('AAPL', '2015-12-26')
5558930000.0
5843082000.0
('AAPL', '2015-09-26')
5753421000.0
6085572000.0
('AAPL', '2015-06-27')
5788922000.0
6136147000.0
('AAPL', '2015-03-28

('ALGN', '2017-03-31')
79904000.0
79831000.0
('ALGN', '2016-12-31')
79856000.0
79998000.0
('ALGN', '2016-09-30')
79920000.0
80173000.0
('ALGN', '2016-06-30')
79891000.0
80358000.0
('ALGN', '2016-03-31')
79831000.0
80459000.0
('ALGN', '2015-12-31')
79998000.0
80754000.0
('ALGN', '2015-09-30')
80173000.0
80924000.0
('ALGN', '2015-06-30')
80358000.0
81073000.0
('ALGN', '2015-03-31')
80459000.0
81120000.0
('ALGN', '2014-12-31')
80754000.0
80551000.0
('ALGN', '2014-09-30')
80924000.0
80592000.0
('ALGN', '2014-06-30')
81073000.0
80909000.0
('ALGN', '2014-03-31')
81120000.0
81248000.0
('ALGN', '2013-12-31')
80551000.0
80529000.0
('ALGN', '2013-09-30')
80592000.0
80356000.0
('ALGN', '2013-06-30')
80909000.0
79810000.0
('ALGN', '2013-03-31')
81248000.0
79235000.0
('ALGN', '2012-12-31')
80529000.0
77988000.0
('ALGN', '2012-09-30')
80356000.0
77735000.0
('ALGN', '2012-06-30')
79810000.0
77369000.0
('ALGN', '2012-03-31')
79235000.0
76844000.0
('ALGN', '2011-12-31')
77988000.0
75825000.0
('ALGN', '

('ANSS', '2019-09-30')
83951000.0
84065000.0
('ANSS', '2019-06-30')
83871000.0
84018000.0
('ANSS', '2019-03-31')
83764000.0
83931000.0
('ANSS', '2018-12-31')
83973000.0
84988000.0
('ANSS', '2018-09-30')
84065000.0
85132000.0
('ANSS', '2018-06-30')
84018000.0
85311000.0
('ANSS', '2018-03-31')
83931000.0
85456000.0
('ANSS', '2017-12-31')
84988000.0
87227000.0
('ANSS', '2017-09-30')
85132000.0
87570000.0
('ANSS', '2017-06-30')
85311000.0
87876000.0
('ANSS', '2017-03-31')
85456000.0
88114000.0
('ANSS', '2016-12-31')
87227000.0
89561000.0
('ANSS', '2016-09-30')
87570000.0
89873000.0
('ANSS', '2016-06-30')
87876000.0
89962000.0
('ANSS', '2016-03-31')
88114000.0
90059000.0
('ANSS', '2015-12-31')
89561000.0
92067000.0
('ANSS', '2015-09-30')
89873000.0
92224000.0
('ANSS', '2015-06-30')
89962000.0
92398000.0
('ANSS', '2015-03-31')
90059000.0
92483000.0
('ANSS', '2014-12-31')
92067000.0
92691000.0
('ANSS', '2014-09-30')
92224000.0
92770000.0
('ANSS', '2014-06-30')
92398000.0
92884000.0
('ANSS', '

('BMRN', '2020-03-31')
179898000.0
178271000.0
('BMRN', '2019-09-30')
178873000.0
176767000.0
('BMRN', '2019-06-30')
178662000.0
176405000.0
('BMRN', '2019-03-31')
178271000.0
175932000.0
('BMRN', '2018-12-31')
178062106.0
174427000.0
('BMRN', '2018-09-30')
176767000.0
174071000.0
('BMRN', '2018-06-30')
176405000.0
173547000.0
('BMRN', '2018-03-31')
175932000.0
172710000.0
('BMRN', '2017-12-31')
174427000.0
165985000.0
('BMRN', '2017-09-30')
174071000.0
163963000.0
('BMRN', '2017-06-30')
173547000.0
162067000.0
('BMRN', '2017-03-31')
172710000.0
161548000.0
('BMRN', '2016-12-31')
165985000.0
160025000.0
('BMRN', '2016-09-30')
163963000.0
159647000.0
('BMRN', '2016-06-30')
162067000.0
159017000.0
('BMRN', '2016-03-31')
161548000.0
157612000.0
('BMRN', '2015-12-31')
160025000.0
146349000.0
('BMRN', '2015-09-30')
159647000.0
145724000.0
('BMRN', '2015-06-30')
159017000.0
145066000.0
('BMRN', '2015-03-31')
157612000.0
143983000.0
('BMRN', '2014-12-31')
146349000.0
137755000.0
('BMRN', '201

('COST', '2020-02-16')
441920000.0
439721000.0
('COST', '2019-11-24')
441818000.0
439157000.0
('COST', '2019-09-01')
439755000.0
438515000.0
('COST', '2019-05-12')
439767000.0
438576000.0
('COST', '2019-02-17')
439721000.0
438494000.0
('COST', '2018-11-25')
439157000.0
437965000.0
('COST', '2018-09-02')
438515000.0
438437000.0
('COST', '2018-05-13')
438576000.0
438650000.0
('COST', '2018-02-18')
438494000.0
438567000.0
('COST', '2017-11-26')
437965000.0
438007000.0
('COST', '2017-09-03')
438437000.0
438585000.0
('COST', '2017-05-07')
438650000.0
438930000.0
('COST', '2017-02-12')
438567000.0
438990000.0
('COST', '2016-11-20')
438007000.0
438342000.0
('COST', '2016-08-28')
438585000.0
439455000.0
('COST', '2016-05-08')
438930000.0
439733000.0
('COST', '2016-02-14')
438990000.0
439567000.0
('COST', '2015-11-22')
438342000.0
438760000.0
('COST', '2015-08-30')
439455000.0
438693000.0
('COST', '2015-05-10')
439733000.0
439058000.0
('COST', '2015-02-15')
439567000.0
438868000.0
('COST', '201

...

('QCOM', '2018-09-30')
1463000000.0
1477000000.0
('QCOM', '2018-06-24')
1479000000.0
1478000000.0
('QCOM', '2018-03-25')
1479000000.0
1478000000.0
('QCOM', '2017-12-24')
1477000000.0
1478000000.0
('QCOM', '2017-09-24')
1477000000.0
1484000000.0
('QCOM', '2017-06-25')
1478000000.0
1487000000.0
('QCOM', '2017-03-26')
1478000000.0
1495000000.0
('QCOM', '2016-12-25')
1478000000.0
1502000000.0
('QCOM', '2016-09-25')
1484000000.0
1618000000.0
('QCOM', '2016-06-26')
1487000000.0
1638000000.0
('QCOM', '2016-03-27')
1495000000.0
1653000000.0
('QCOM', '2015-12-27')
1502000000.0
1661000000.0
('QCOM', '2015-09-27')
1618000000.0
1683000000.0
('QCOM', '2015-06-28')
1638000000.0
1686000000.0
('QCOM', '2015-03-29')
1653000000.0
1688000000.0
('QCOM', '2014-12-28')
1661000000.0
1688000000.0
('QCOM', '2014-09-28')
1683000000.0
1715000000.0
('QCOM', '2014-06-29')
1686000000.0
1720000000.0
('QCOM', '2014-03-30')
1688000000.0
1716000000.0
('QCOM', '2013-12-29')
1688000000.0
1709000000.0
('QCOM', '2013-09-29

('SIRI', '2020-03-31')
4405000000.0
4571000000.0
('SIRI', '2019-12-31')
4501000000.0
4461827000.0
('SIRI', '2019-09-30')
4529000000.0
4482249000.0
('SIRI', '2019-06-30')
4569000000.0
4486620000.0
('SIRI', '2019-03-31')
4571000000.0
4491362000.0
('SIRI', '2018-12-31')
4461827000.0
4637553000.0
('SIRI', '2018-09-30')
4482249000.0
4660041000.0
('SIRI', '2018-06-30')
4486620000.0
4681223000.0
('SIRI', '2018-03-31')
4491362000.0
4710340000.0
('SIRI', '2017-12-31')
4637553000.0
4917050000.0
('SIRI', '2017-09-30')
4660041000.0
4957820000.0
('SIRI', '2017-06-30')
4681223000.0
5002070000.0
('SIRI', '2017-03-31')
4710340000.0
5065319000.0
('SIRI', '2016-12-31')
4917050000.0
5375707000.0
('SIRI', '2016-09-30')
4957820000.0
5436378000.0
('SIRI', '2016-06-30')
5002070000.0
5506818000.0
('SIRI', '2016-03-31')
5065319000.0
5570748000.0
('SIRI', '2015-12-31')
5375707000.0
5788944000.0
('SIRI', '2015-09-30')
5436378000.0
5860248000.0
('SIRI', '2015-06-30')
5506818000.0
5979273000.0
('SIRI', '2015-03-31

('TTWO', '2018-09-30')
113339000.0
107232000.0
('TTWO', '2018-06-30')
112941000.0
105494000.0
('TTWO', '2018-03-31')
110210000.0
91921000.0
('TTWO', '2017-12-31')
109010000.0
86796000.0
('TTWO', '2017-09-30')
107232000.0
84990000.0
('TTWO', '2017-06-30')
105494000.0
84588000.0
('TTWO', '2017-03-31')
91921000.0
83417000.0
('TTWO', '2016-12-31')
86796000.0
83338000.0
('TTWO', '2016-09-30')
84990000.0
83280000.0
('TTWO', '2016-06-30')
84588000.0
82833000.0
('TTWO', '2016-03-31')
83417000.0
80367000.0
('TTWO', '2015-12-31')
83338000.0
80128000.0
('TTWO', '2015-09-30')
83280000.0
79862000.0
('TTWO', '2015-06-30')
82833000.0
79369000.0
('TTWO', '2015-03-31')
80367000.0
95347000.0
('TTWO', '2014-12-31')
80128000.0
86970000.0
('TTWO', '2014-09-30')
79862000.0
87907000.0
('TTWO', '2014-06-30')
79369000.0
86992000.0
('TTWO', '2014-03-31')
95347000.0
85581000.0
('TTWO', '2013-12-31')
86970000.0
85382000.0
('TTWO', '2013-09-30')
87907000.0
85197000.0
('TTWO', '2013-06-30')
86992000.0
85010000.0
('

('VRSN', '2011-09-30')
167492000.0
179240000.0
('VRSN', '2011-06-30')
169751000.0
182121000.0
('VRSN', '2011-03-31')
170193000.0
183174000.0
('VRSN', '2010-12-31')
177534000.0
192345267.0
('VRSN', '2010-09-30')
179240000.0
192868550.0
('VRSN', '2010-06-30')
182121000.0
192649000.0
('VRTX', '2020-03-31')
259815000.0
255695000.0
('VRTX', '2019-12-31')
256728000.0
254292000.0
('VRTX', '2019-09-30')
256289000.0
254096000.0
('VRTX', '2019-06-30')
255941000.0
253685000.0
('VRTX', '2019-03-31')
255695000.0
253231000.0
('VRTX', '2018-12-31')
254292000.0
248858000.0
('VRTX', '2018-09-30')
254096000.0
247963000.0
('VRTX', '2018-06-30')
253685000.0
246782000.0
('VRTX', '2018-03-31')
253231000.0
246024000.0
('VRTX', '2017-12-31')
248858000.0
244685000.0
('VRTX', '2017-09-30')
247963000.0
244529000.0
('VRTX', '2017-06-30')
246782000.0
244124000.0
('VRTX', '2017-03-31')
246024000.0
243831000.0
('VRTX', '2016-12-31')
244685000.0
241312000.0
('VRTX', '2016-09-30')
244529000.0
240749000.0
('VRTX', '201

('XLNX', '2017-04-01')
252301000.0
257184000.0
('XLNX', '2016-12-31')
252811000.0
257491000.0
('XLNX', '2016-10-01')
253056000.0
257744000.0
('XLNX', '2016-07-02')
252901000.0
258021000.0
('XLNX', '2016-04-02')
257184000.0
265480000.0
('XLNX', '2016-01-02')
257491000.0
266299000.0
('XLNX', '2015-09-26')
257744000.0
267098000.0
('XLNX', '2015-06-27')
258021000.0
267648000.0
('XLNX', '2015-03-28')
265480000.0
266431000.0
('XLNX', '2014-12-27')
266299000.0
266068000.0
('XLNX', '2014-09-27')
267098000.0
265350000.0
('XLNX', '2014-06-28')
267648000.0
264153000.0
('XLNX', '2014-03-29')
266431000.0
261652000.0
('XLNX', '2013-12-28')
266068000.0
261723000.0
('XLNX', '2013-09-28')
265350000.0
262143000.0
('XLNX', '2013-06-29')
264153000.0
263055000.0
('XLNX', '2013-03-30')
261652000.0
263783000.0
('XLNX', '2012-12-29')
261723000.0
264183000.0
('XLNX', '2012-09-29')
262143000.0
264853000.0
('XLNX', '2012-06-30')
263055000.0
265313000.0


In [21]:
# Question 8. Change in Gross Margin
"""
Compare this year's gross margin (Gross Profit divided by sales) to last year's.
Score 1 if this year's gross margin is higher, 0 if it's lower.
"""
piotroski_dfs8 = pd.DataFrame(columns=['ticker','date','score8'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:8+item]
        
        # Calculate if no data is missing
        if((len(temp33.index) == 8) and (temp33['Gross Profit'].all()) and (temp33['Revenue'].all())):
            
            # Calculating 8th score
            gm_this_year = temp33['Gross Profit'][0:4].sum() / temp33['Revenue'][0:4].sum()
            gm_last_year = temp33['Gross Profit'][4:8].sum() / temp33['Revenue'][4:8].sum()
            
            print(new_df.index[item])
            print(gm_this_year)
            print(gm_last_year)
            
            if(gm_this_year > gm_last_year):
                piotroski_dfs8 = piotroski_dfs8.append({'ticker':ticker, 'date':new_df.index[item][1], 'score8':1}, ignore_index=True)
            else:
                piotroski_dfs8 = piotroski_dfs8.append({'ticker':ticker, 'date':new_df.index[item][1], 'score8':0}, ignore_index=True)
        else:
            piotroski_dfs8 = piotroski_dfs8.append({'ticker':ticker, 'date':new_df.index[item][1], 'score8':None}, ignore_index=True)

('AAPL', '2020-03-28')
0.38109791365805784
0.38050601570660375
('AAPL', '2019-12-28')
0.3794712402356519
0.38208491965200375
('AAPL', '2019-09-28')
0.3781776810903472
0.38343718820007905
('AAPL', '2019-06-29')
0.3789579746288132
0.3826711690183881
('AAPL', '2019-03-30')
0.38050601570660375
0.38296883399281373
('AAPL', '2018-12-29')
0.38208491965200375
0.3843278589825066
('AAPL', '2018-09-29')
0.38343718820007905
0.38469860491899105
('AAPL', '2018-06-30')
0.3826711690183881
0.38507966193452553
('AAPL', '2018-03-31')
0.38296883399281373
0.3841384034074672
('AAPL', '2017-12-30')
0.3843278589825066
0.38518600023840305
('AAPL', '2017-09-30')
0.38469860491899105
0.3907595564809705
('AAPL', '2017-07-01')
0.38507966193452553
0.39492845729227194
('AAPL', '2017-04-01')
0.3841384034074672
0.39806183664051686
('AAPL', '2016-12-31')
0.38518600023840305
0.40133113180247504
('AAPL', '2016-09-24')
0.3907595564809705
0.4005990201741437
('AAPL', '2016-06-25')
0.39492845729227194
0.3971123800353932
('AAP

('ALGN', '2019-09-30')
0.7223981014644512
0.7458688797844422
('ALGN', '2019-06-30')
0.7260351947957021
0.7517605938435451
('ALGN', '2019-03-31')
0.7325121154811144
0.7554581386453466
('ALGN', '2018-12-31')
0.7362689499881007
0.7580678329836916
('ALGN', '2018-09-30')
0.7458688797844422
0.7576632005072501
('ALGN', '2018-06-30')
0.7517605938435451
0.7556771070913884
('ALGN', '2018-03-31')
0.7554581386453466
0.7557931211164617
('ALGN', '2017-12-31')
0.7580678329836916
0.7549899340108198
('ALGN', '2017-09-30')
0.7576632005072501
0.7550590148749148
('ALGN', '2017-06-30')
0.7556771070913884
0.7571179098387516
('ALGN', '2017-03-31')
0.7557931211164617
0.7557069019997291
('ALGN', '2016-12-31')
0.7549899340108198
0.7570911877902177
('ALGN', '2016-09-30')
0.7550590148749148
0.7593443186984677
('ALGN', '2016-06-30')
0.7571179098387516
0.7605552415049306
('ALGN', '2016-03-31')
0.7557069019997291
0.7602198967260648
('ALGN', '2015-12-31')
0.7570911877902177
0.75945739070154
('ALGN', '2015-09-30')
0.7

('AMZN', '2020-03-31')
0.40638395539264327
0.41070607961249456
('AMZN', '2019-12-31')
0.4099001147860061
0.4024741612885219
('AMZN', '2019-09-30')
0.41106117851801904
0.3986839068234996
('AMZN', '2019-06-30')
0.41266270733903826
0.38781981981981983
('AMZN', '2019-03-31')
0.41070607961249456
0.37767736058055634
('AMZN', '2018-12-31')
0.4024741612885219
0.3706835482891615
('AMZN', '2018-09-30')
0.3986839068234996
0.3645953559948869
('AMZN', '2018-06-30')
0.38781981981981983
0.3598025632144094
('AMZN', '2018-03-31')
0.37767736058055634
0.3558457772509521
('AMZN', '2017-12-31')
0.3706835482891615
0.35093060366064405
('AMZN', '2017-09-30')
0.3645953559948869
0.34647207269147534
('AMZN', '2017-06-30')
0.3598025632144094
0.3439574923116457
('AMZN', '2017-03-31')
0.3558457772509521
0.3376477953040549
('AMZN', '2016-12-31')
0.35093060366064405
0.33040203353083003
('AMZN', '2016-09-30')
0.34647207269147534
0.32415397462918044
('AMZN', '2016-06-30')
0.3439574923116457
0.31264612558450233
('AMZN',

('BIIB', '2019-12-31')
0.8639996105133573
0.8649882181537066
('BIIB', '2019-09-30')
0.8597053306072551
0.8611866763390158
('BIIB', '2019-06-30')
0.8559104023650137
0.8643454212112738
('BIIB', '2019-03-31')
0.8571997451417649
0.8657011505204736
('BIIB', '2018-12-31')
0.8649882181537066
0.8671978751660027
('BIIB', '2018-09-30')
0.8611866763390158
0.8733581667215704
('BIIB', '2018-06-30')
0.8643454212112738
0.8680367674043475
('BIIB', '2018-03-31')
0.8657011505204736
0.8655735430558326
('BIIB', '2017-12-31')
0.8671978751660027
0.8708423590245266
('BIIB', '2017-09-30')
0.8733581667215704
0.8745616730564527
('BIIB', '2017-06-30')
0.8680367674043475
0.8820852458958044
('BIIB', '2017-03-31')
0.8655735430558326
0.8865206480427249
('BIIB', '2016-12-31')
0.8708423590245266
0.8847618870659061
('BIIB', '2016-09-30')
0.8745616730564527
0.8858669062470451
('BIIB', '2016-06-30')
0.8820852458958044
0.883631917190534
('BIIB', '2016-03-31')
0.8865206480427249
0.8811072004878099
('BIIB', '2015-12-31')
0.

('CERN', '2017-04-01')
0.9139530099064329
0.9062772794419827
('CERN', '2016-12-31')
0.9140897905606891
0.9027550202055605
('CERN', '2016-10-01')
0.9115582098063025
0.9013624029095758
('CERN', '2016-07-02')
0.9078012950912528
0.9044121466998399
('CERN', '2016-04-02')
0.9062772794419827
0.9057907915012351
('CERN', '2016-01-02')
0.9027550202055605
0.9076942654119386
('CERN', '2015-10-03')
0.9013624029095758
0.9093114303178484
('CERN', '2015-07-04')
0.9044121466998399
0.9064456190355925
('CERN', '2015-04-04')
0.9057907915012351
0.9051547349012429
('CERN', '2015-01-03')
0.9076942654119386
0.8961181112208958
('CERN', '2014-09-27')
0.9093114303178484
0.88433628093102
('CERN', '2014-06-28')
0.9064456190355925
0.8691195679441303
('CERN', '2014-03-29')
0.9051547349012429
0.8550490653056501
('CERN', '2013-12-28')
0.8961181112208958
0.8396299892400343
('CERN', '2013-09-28')
0.88433628093102
0.8403145006881453
('CERN', '2013-06-29')
0.8691195679441303
0.8682296795082884
('CERN', '2013-03-30')
0.855

...

('QCOM', '2011-03-27')
0.806444099378882
0.8249981366922561
('REGN', '2020-03-31')
0.9536078598461114
0.9737152398708457
('REGN', '2019-12-31')
0.9539257827402905
0.9731775645228586
('REGN', '2019-09-30')
0.9609286488297816
0.9703488362959414
('REGN', '2019-06-30')
0.9706075605589382
0.9670600859925964
('REGN', '2019-03-31')
0.9737152398708457
0.965291560815411
('REGN', '2018-12-31')
0.9731775645228586
0.9655144462228725
('REGN', '2018-09-30')
0.9703488362959414
0.9647776250873046
('REGN', '2018-06-30')
0.9670600859925964
0.9660391332713019
('REGN', '2018-03-31')
0.965291560815411
0.9644606713294523
('REGN', '2017-12-31')
0.9655144462228725
0.9599574276087266
('REGN', '2017-09-30')
0.9647776250873046
0.9532580097922999
('REGN', '2017-06-30')
0.9660391332713019
0.9444036827887081
('REGN', '2017-03-31')
0.9644606713294523
0.9372996179225767
('REGN', '2016-12-31')
0.9599574276087266
0.9411018469060327
('REGN', '2016-09-30')
0.9532580097922999
0.9452252900487923
('REGN', '2016-06-30')
0.94

('SIRI', '2020-03-31')
0.5647338165458635
0.8147125459517655
('SIRI', '2019-12-31')
0.5659481652553247
0.9296988991961449
('SIRI', '2019-09-30')
0.6425045808552073
0.9447665849223464
('SIRI', '2019-06-30')
0.7291859375960741
0.9596395568151432
('SIRI', '2019-03-31')
0.8147125459517655
0.9935285266605705
('SIRI', '2018-12-31')
0.9296988991961449
0.9934659618232119
('SIRI', '2018-09-30')
0.9447665849223464
0.9931937563166423
('SIRI', '2018-06-30')
0.9596395568151432
0.9927889720177723
('SIRI', '2018-03-31')
0.9935285266605705
0.9925610671517546
('SIRI', '2017-12-31')
0.9934659618232119
0.9918516628730651
('SIRI', '2017-09-30')
0.9931937563166423
0.9912666423371934
('SIRI', '2017-06-30')
0.9927889720177723
0.9911594828788901
('SIRI', '2017-03-31')
0.9925610671517546
0.99069141280806
('SIRI', '2016-12-31')
0.9918516628730651
0.990651322149522
('SIRI', '2016-09-30')
0.9912666423371934
0.9901231034136762
('SIRI', '2016-06-30')
0.9911594828788901
0.989812599905474
('SIRI', '2016-03-31')
0.990

('TMUS', '2014-12-31')
0.674570423488026
0.7143325143325143
('TMUS', '2014-09-30')
0.683110812055105
0.7206123444882194
('TMUS', '2014-06-30')
0.686128739800544
0.7378863142290579
('TMUS', '2014-03-31')
0.7059434504982165
0.7224180999497433
('TMUS', '2013-12-31')
0.7143325143325143
0.7177522965813665
('TMUS', '2013-09-30')
0.7206123444882194
0.7335258783465
('TMUS', '2013-06-30')
0.7378863142290579
0.7151583690955373
('TMUS', '2013-03-31')
0.7224180999497433
0.6979068260171475
('TMUS', '2012-12-31')
0.7177522965813665
0.7030159785220146
('TMUS', '2012-09-30')
0.7335258783465
0.7039717477516946
('TMUS', '2012-06-30')
0.7151583690955373
0.7112234510125847
('TMUS', '2012-03-31')
0.6979068260171475
0.7144219023392158
('TMUS', '2011-12-31')
0.7030159785220146
0.6642098504401713
('TMUS', '2011-09-30')
0.7039717477516946
0.6036213853485229
('TMUS', '2011-06-30')
0.7112234510125847
0.5427859461875376
('TMUS', '2011-03-31')
0.7144219023392158
0.416766579432769
('TSLA', '2020-03-31')
0.182051397

('UAL', '2015-06-30')
1.0
1.0
('UAL', '2015-03-31')
1.0
1.0
('UAL', '2014-12-31')
1.0
1.0
('UAL', '2014-09-30')
1.0
1.0
('UAL', '2014-06-30')
1.0
1.0
('UAL', '2014-03-31')
1.0
1.0
('UAL', '2013-12-31')
1.0
1.0
('UAL', '2013-09-30')
1.0
1.0
('UAL', '2013-06-30')
1.0
1.0
('UAL', '2013-03-31')
1.0
1.0
('UAL', '2012-12-31')
1.0
1.0
('UAL', '2012-09-30')
1.0
1.0
('UAL', '2012-06-30')
1.0
1.0
('UAL', '2012-03-31')
1.0
1.0
('UAL', '2011-12-31')
1.0
0.9215507826720059
('UAL', '2011-09-30')
1.0
0.8777873439517865
('UAL', '2011-06-30')
1.0
0.7492927387848579
('UAL', '2011-03-31')
1.0
0.5344980752146876
('ULTA', '2020-02-01')
0.36240056187642505
0.358709111658179
('ULTA', '2019-11-02')
0.3623509152033584
0.3563522792205395
('ULTA', '2019-08-03')
0.3613972107150423
0.3561105388898422
('ULTA', '2019-05-04')
0.3605340214404993
0.35684755223615944
('ULTA', '2019-02-02')
0.358709111658179
0.35632710715224014
('ULTA', '2018-11-03')
0.3563522792205395
0.35881941308601667
('ULTA', '2018-08-04')
0.3561105

('WBA', '2014-05-31')
0.2847255433338647
0.29100618088550645
('WBA', '2014-02-28')
0.2859626893837278
0.2903740747019269
('WBA', '2013-11-30')
0.28911648231599074
0.28727822352808224
('WBA', '2013-08-31')
0.2924380685988064
0.2839752627978725
('WBA', '2013-05-31')
0.29100618088550645
0.28370124229597254
('WBA', '2013-02-28')
0.2903740747019269
0.2832143931315451
('WBA', '2012-11-30')
0.28727822352808224
0.2829020370700166
('WBA', '2012-08-31')
0.2839752627978725
0.28388562562340686
('WBA', '2012-05-31')
0.28370124229597254
0.2843698566545219
('WBA', '2012-02-29')
0.2832143931315451
0.283344060645069
('WBA', '2011-11-30')
0.2829020370700166
0.2833771929824561
('WBA', '2011-08-31')
0.28388562562340686
0.2814595075645209
('WBA', '2011-05-31')
0.2843698566545219
0.27968544820611896
('WBA', '2011-02-28')
0.283344060645069
0.27948026477077714
('WDAY', '2020-01-31')
0.8422313863019149
0.8387512490344343
('WDAY', '2019-10-31')
0.8601892406967803
0.8738073540384828
('WDAY', '2019-07-31')
0.8525

In [22]:
# Question 9. Change in asset turnover
"""
Compare this year's asset turnover (total sales for the year divided by total assets at the beginning of the year) to last year's asset turnover ratio.
Score 1 if this year's asset turnover ratio is higher, 0 if it's lower
"""
piotroski_dfs9 = pd.DataFrame(columns=['ticker','date','score9'])
for ticker, new_df in joined_data.groupby(level=0):

    for item in range(len(new_df)):
        
        # Take 4 quarterly numbers for Trailing Twelve Months
        temp33 = new_df.iloc[item:9+item]
        
        # Calculate if no data is missing
        if((len(temp33.index) == 9) and (temp33['Revenue'].all()) and (temp33['Total assets'][4] != 0) 
           and (temp33['Total assets'][8] != 0)):
            
            # Calculating 9th score
            ast_this_year = temp33['Revenue'][0:4].sum() / temp33['Total assets'][4]
            ast_last_year = temp33['Revenue'][4:8].sum() / temp33['Total assets'][8]
            
            print(new_df.index[item])
            print(ast_this_year)
            print(ast_last_year)
            
            if(ast_this_year > ast_last_year):
                piotroski_dfs9 = piotroski_dfs9.append({'ticker':ticker, 'date':new_df.index[item][1], 'score9':1}, ignore_index=True)
            else:
                piotroski_dfs9 = piotroski_dfs9.append({'ticker':ticker, 'date':new_df.index[item][1], 'score9':0}, ignore_index=True)
        else:
            piotroski_dfs9 = piotroski_dfs9.append({'ticker':ticker, 'date':new_df.index[item][1], 'score9':None}, ignore_index=True)

('AAPL', '2020-03-28')
0.7835747577471214
0.7033703217941671
('AAPL', '2019-12-28')
0.7162681051806303
0.6431068304842255
('AAPL', '2019-09-28')
0.711392439674619
0.7076513579115365
('AAPL', '2019-06-29')
0.7417990418016192
0.7395537889695891
('AAPL', '2019-03-30')
0.7033703217941671
0.739591429220523
('AAPL', '2018-12-29')
0.6431068304842255
0.7222784252025573
('AAPL', '2018-09-29')
0.7076513579115365
0.7126017296369752
('AAPL', '2018-06-30')
0.7395537889695891
0.7313662868698503
('AAPL', '2018-03-31')
0.739591429220523
0.7221539781902993
('AAPL', '2017-12-30')
0.7222784252025573
0.7437091692693771
('AAPL', '2017-09-30')
0.7126017296369752
0.7423565903215035
('AAPL', '2017-07-01')
0.7313662868698503
0.8064696816046802
('AAPL', '2017-04-01')
0.7221539781902993
0.8711340995581828
('AAPL', '2016-12-31')
0.7437091692693771
0.897263778475261
('AAPL', '2016-09-24')
0.7423565903215035
1.0080918223422288
('AAPL', '2016-06-25')
0.8064696816046802
1.0081655581520763
('AAPL', '2016-03-26')
0.871

('ALGN', '2018-12-31')
1.1061030814644155
1.0553392863665894
('ALGN', '2018-09-30')
1.091994512326942
0.9899918536632392
('ALGN', '2018-06-30')
1.1177182377108628
0.9898839022339649
('ALGN', '2018-03-31')
1.096418608189034
0.9535716651760575
('ALGN', '2017-12-31')
1.0553392863665894
0.9320242043856856
('ALGN', '2017-09-30')
0.9899918536632392
0.9419996257741565
('ALGN', '2017-06-30')
0.9898839022339649
0.9197078304965773
('ALGN', '2017-03-31')
0.9535716651760575
0.8616449744993461
('ALGN', '2016-12-31')
0.9320242043856856
0.8557576591831757
('ALGN', '2016-09-30')
0.9419996257741565
0.8581877465624931
('ALGN', '2016-06-30')
0.9197078304965773
0.8990516504354367
('ALGN', '2016-03-31')
0.8616449744993461
0.8796939166095518
('ALGN', '2015-12-31')
0.8557576591831757
0.9152866020066166
('ALGN', '2015-09-30')
0.8581877465624931
0.9764741058421561
('ALGN', '2015-06-30')
0.8990516504354367
1.0171630955797322
('ALGN', '2015-03-31')
0.8796939166095518
0.9463362914718862
('ALGN', '2014-12-31')
0.9

('AMZN', '2015-03-31')
2.5289847101528986
2.7530394333439054
('AMZN', '2014-12-31')
2.215891829975846
2.286960528336661
('AMZN', '2014-09-30')
2.675559461410502
3.0714285714285716
('AMZN', '2014-06-30')
2.759983796374439
3.179906764342118
('AMZN', '2014-03-31')
2.7530394333439054
3.1455823786813513
('AMZN', '2013-12-31')
2.286960528336661
2.416844687079674
('AMZN', '2013-09-30')
3.0714285714285716
3.004933347328645
('AMZN', '2013-06-30')
3.179906764342118
3.0280363413410623
('AMZN', '2013-03-31')
3.1455823786813513
3.044959128065395
('AMZN', '2012-12-31')
2.416844687079674
2.557695376921849
('AMZN', '2012-09-30')
3.004933347328645
3.0781669255754838
('AMZN', '2012-06-30')
3.0280363413410623
3.2489311930305718
('AMZN', '2012-03-31')
3.044959128065395
3.066766317887394
('AMZN', '2011-12-31')
2.557695376921849
2.476218055455006
('AMZN', '2011-09-30')
3.0781669255754838
3.4303388319215338
('AMZN', '2011-06-30')
3.2489311930305718
3.734983713355049
('ANSS', '2020-03-31')
0.4459162882610283


('BKNG', '2017-06-30')
0.583270475136853
0.575951002581377
('BKNG', '2017-03-31')
0.600739772885568
0.5659145926861654
('BKNG', '2016-12-31')
0.6166849257271932
0.6173788096204942
('BKNG', '2016-09-30')
0.6256221422264647
0.5969350740695373
('BKNG', '2016-06-30')
0.575951002581377
0.7584000031723255
('BKNG', '2016-03-31')
0.5659145926861654
0.7990674593413308
('BKNG', '2015-12-31')
0.6173788096204942
0.808272615338658
('BKNG', '2015-09-30')
0.5969350740695373
0.7795611668200797
('BKNG', '2015-06-30')
0.7584000031723255
0.7678269115451319
('BKNG', '2015-03-31')
0.7990674593413308
1.0405490664140868
('BKNG', '2014-12-31')
0.808272615338658
1.0340293424003562
('BKNG', '2014-09-30')
0.7795611668200797
1.0361838068683036
('BKNG', '2014-06-30')
0.7678269115451319
1.0829637308829971
('BKNG', '2014-03-31')
1.0405490664140868
1.085000727493787
('BKNG', '2013-12-31')
1.0340293424003562
1.3249538931832932
('BKNG', '2013-09-30')
1.0361838068683036
1.326429084176872
('BKNG', '2013-06-30')
1.0829637

('CHTR', '2017-12-31')
0.2789416839407783
0.7376894902838539
('CHTR', '2017-09-30')
0.27706401069195485
0.5760312423724676
('CHTR', '2017-06-30')
0.27368879654143907
0.7883249610254633
('CHTR', '2017-03-31')
0.9040815319316948
0.40647275706677594
('CHTR', '2016-12-31')
0.7376894902838539
0.39731160896130346
('CHTR', '2016-09-30')
0.5760312423724676
0.4583293556085919
('CHTR', '2016-06-30')
0.7883249610254633
0.5434394611088721
('CHTR', '2016-03-31')
0.40647275706677594
0.5358154593282072
('CHTR', '2015-12-31')
0.39731160896130346
0.5266261925411969
('CHTR', '2015-09-30')
0.4583293556085919
0.5157101449275362
('CHTR', '2015-06-30')
0.5434394611088721
0.5598178202578742
('CHTR', '2015-03-31')
0.5358154593282072
0.5396764499008888
('CHTR', '2014-12-31')
0.5266261925411969
0.522789922430925
('CHTR', '2014-09-30')
0.5157101449275362
0.4800872885979269
('CHTR', '2014-06-30')
0.5598178202578742
0.4939240018002958
('CHTR', '2014-03-31')
0.5396764499008888
0.4893356530704298
('CHTR', '2013-12-3

...

('REGN', '2016-06-30')
1.0269511395039066
0.9827168252568373
('REGN', '2016-03-31')
1.0563029391176477
0.9395880132352021
('REGN', '2015-12-31')
1.0598944632598513
0.9554539407315387
('REGN', '2015-09-30')
1.018334666428127
0.9951617454074454
('REGN', '2015-06-30')
0.9827168252568373
1.0507177729591268
('REGN', '2015-03-31')
0.9395880132352021
1.0065127473749977
('REGN', '2014-12-31')
0.9554539407315387
1.011658311263212
('REGN', '2014-09-30')
0.9951617454074454
1.1495349614450068
('REGN', '2014-06-30')
1.0507177729591268
1.1935471698113207
('REGN', '2014-03-31')
1.0065127473749977
1.1749371740661656
('REGN', '2013-12-31')
1.011658311263212
1.0414737874390951
('REGN', '2013-09-30')
1.1495349614450068
1.1056131495344501
('REGN', '2013-06-30')
1.1935471698113207
0.7278863348056672
('REGN', '2013-03-31')
1.1749371740661656
0.5263079997728739
('REGN', '2012-12-31')
1.0414737874390951
0.40922609212874234
('REGN', '2012-09-30')
1.1056131495344501
0.4844079101644361
('REGN', '2012-06-30')
0.7

('SNPS', '2016-07-31')
0.47960949315014295
0.4827498955429227
('SNPS', '2016-04-30')
0.4870672118956294
0.47364410674534746
('SNPS', '2016-01-31')
0.48661565232461146
0.505840908592142
('SNPS', '2015-10-31')
0.4695239178146619
0.4720125443485622
('SNPS', '2015-07-31')
0.4827498955429227
0.4718079073385579
('SNPS', '2015-04-30')
0.47364410674534746
0.48414789572196165
('SNPS', '2015-01-31')
0.505840908592142
0.4889952757660278
('SNPS', '2014-10-31')
0.4720125443485622
0.47308986087563676
('SNPS', '2014-07-31')
0.4718079073385579
0.4940341475177107
('SNPS', '2014-04-30')
0.48414789572196165
0.5000301777388702
('SNPS', '2014-01-31')
0.4889952757660278
0.5473349499848439
('SNPS', '2013-10-31')
0.47308986087563676
0.5214858401334705
('SNPS', '2013-07-31')
0.4940341475177107
0.5023804873067395
('SNPS', '2013-04-30')
0.5000301777388702
0.5095503337156395
('SNPS', '2013-01-31')
0.5473349499848439
0.49684155295136945
('SNPS', '2012-10-31')
0.5214858401334705
0.46725204401831594
('SNPS', '2012-0

('TTWO', '2016-12-31')
0.6133046345293188
0.5930296945006207
('TTWO', '2016-09-30')
0.6303110408901472
0.8728660374775103
('TTWO', '2016-06-30')
0.6163183434186119
0.7667987779088109
('TTWO', '2016-03-31')
0.6336327372148267
0.601755916493946
('TTWO', '2015-12-31')
0.5930296945006207
0.49263681837619644
('TTWO', '2015-09-30')
0.8728660374775103
0.855427765894672
('TTWO', '2015-06-30')
0.7667987779088109
1.6112961421996332
('TTWO', '2015-03-31')
0.601755916493946
1.8394868211097015
('TTWO', '2014-12-31')
0.49263681837619644
2.063276350857847
('TTWO', '2014-09-30')
0.855427765894672
0.8643531416747872
('TTWO', '2014-06-30')
1.6112961421996332
1.0539297405643455
('TTWO', '2014-03-31')
1.8394868211097015
1.0565986356680328
('TTWO', '2013-12-31')
2.063276350857847
0.9320390392391347
('TTWO', '2013-09-30')
0.8643531416747872
0.9481884633378974
('TTWO', '2013-06-30')
1.0539297405643455
0.7150243578718922
('TTWO', '2013-03-31')
1.0565986356680328
0.8499103080401663
('TTWO', '2012-12-31')
0.932

('VRSN', '2016-06-30')
0.4305296219186949
0.4442800716772423
('VRSN', '2016-03-31')
0.4152378907163757
0.39080360657006785
('VRSN', '2015-12-31')
0.49161731695670113
0.37963376725583264
('VRSN', '2015-09-30')
0.47234194458513146
0.42912061240772875
('VRSN', '2015-06-30')
0.4442800716772423
0.391504257099754
('VRSN', '2015-03-31')
0.39080360657006785
0.47193755241746826
('VRSN', '2014-12-31')
0.37963376725583264
0.46792641465888574
('VRSN', '2014-09-30')
0.42912061240772875
0.47883337770463236
('VRSN', '2014-06-30')
0.391504257099754
0.47862590781576336
('VRSN', '2014-03-31')
0.47193755241746826
0.4803124136905394
('VRSN', '2013-12-31')
0.46792641465888574
0.47063995444404877
('VRSN', '2013-09-30')
0.47883337770463236
0.5109714672478705
('VRSN', '2013-06-30')
0.47862590781576336
0.45693837328003273
('VRSN', '2013-03-31')
0.4803124136905394
0.34162017297669317
('VRSN', '2012-12-31')
0.47063995444404877
0.3158658366632488
('VRSN', '2012-09-30')
0.5109714672478705
0.2548092016571355
('VRSN

('XLNX', '2020-03-28')
0.4671586932197165
0.6050691033681476
('XLNX', '2019-12-28')
0.604108603105571
0.5671398594309459
('XLNX', '2019-09-28')
0.6433011833234387
0.5435815021318465
('XLNX', '2019-06-29')
0.6378198889379474
0.5126589768318137
('XLNX', '2019-03-30')
0.6050691033681476
0.5355947391558584
('XLNX', '2018-12-29')
0.5671398594309459
0.511879151944632
('XLNX', '2018-09-29')
0.5435815021318465
0.4913519164044871
('XLNX', '2018-06-30')
0.5126589768318137
0.4850922820069116
('XLNX', '2018-03-31')
0.5355947391558584
0.4870941296918987
('XLNX', '2017-12-30')
0.511879151944632
0.4717984744057816
('XLNX', '2017-09-30')
0.4913519164044871
0.46857402844377194
('XLNX', '2017-07-01')
0.4850922820069116
0.45566710859763576
('XLNX', '2017-04-01')
0.4870941296918987
0.4519909392790827
('XLNX', '2016-12-31')
0.4717984744057816
0.4495670983340671
('XLNX', '2016-10-01')
0.46857402844377194
0.44586293589662335
('XLNX', '2016-07-02')
0.45566710859763576
0.4577797281835235
('XLNX', '2016-04-02')

In [37]:
pssd1 = piotroski_dfs1.set_index(['ticker','date'])
pssd2 = piotroski_dfs2.set_index(['ticker','date'])
pssd3 = piotroski_dfs3.set_index(['ticker','date'])
pssd4 = piotroski_dfs4.set_index(['ticker','date'])
pssd5 = piotroski_dfs5.set_index(['ticker','date'])
pssd6 = piotroski_dfs6.set_index(['ticker','date'])
pssd7 = piotroski_dfs7.set_index(['ticker','date'])
pssd8 = piotroski_dfs8.set_index(['ticker','date'])
pssd9 = piotroski_dfs9.set_index(['ticker','date'])

piotrs_joined = pssd1.join(pssd2, how='inner')
piotrs_joined = piotrs_joined.join(pssd3, how='inner')
piotrs_joined = piotrs_joined.join(pssd4, how='inner')
piotrs_joined = piotrs_joined.join(pssd5, how='inner')
piotrs_joined = piotrs_joined.join(pssd6, how='inner')
piotrs_joined = piotrs_joined.join(pssd7, how='inner')
piotrs_joined = piotrs_joined.join(pssd8, how='inner')
piotrs_joined = piotrs_joined.join(pssd9, how='inner')

piotrs_joined.sort_index(level=0, inplace=True)
piotrs_joined.dropna(how='any', inplace=True)
print(len(piotrs_joined.index))
print(piotrs_joined)

2268


In [40]:
# clean dataset if any of the columns is na
piotrs_joined.to_csv('Pscorovmerged.csv', sep=',')

In [44]:
# Importing the data to csv file
piotr_joined.to_csv('PiotrSkorov.csv', sep=',')